# Proyecto Segundo Bimestre: Desarrollo de un sistema de generación aumentada por recuperación RAG 

Nombres:
Jostin Vega,
Gabriela Salazar,
Vickiann Jiménez

El presente proyecto busca desarrollar un sistema RAG que permita la recuperación de información de los 16 candidatos presidenciales y sus propuestas de campaña previo a los comisios electorales en Ecuador del 09 de febrero del 2025
A continuación, se presenta los códigos utilizados para el desarrollo del proyecto. 

## 0. Construcción del Corpus
### Obtención de entrevistas
Extracción del audio de las entrevistas realizadas a los candidatos presidenciales. Los audios fueron extraidos a partir de videos en Youtube para posteriormente se pasados a texto, ser procesados y formar parte del corpus del presente proyecto.

#### Instalaciones necesarias

In [ ]:
#Instalación de Whisper, modelo de reconocimiento automático de voz desarrollado por OpenAI. 
# Se utiliza la la transcripción y traducción de audio a texto
pip install git+https://github.com/openai/whisper

  Cloning https://github.com/openai/whisper to /tmp/pip-req-build-bo219cr1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper /tmp/pip-req-build-bo219cr1
  Resolved https://github.com/openai/whisper to commit 517a43ecd132a2089d85f4ebc044728a71d49f6e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [ ]:
# Instalación de la librería Pytube, 
# biblioteca de Python que permite la descarga de videos de YouTube 
# y la extracción de sus audios.
pip install pytube

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 3.5 MB/s eta 0:00:00


In [ ]:
pip install --upgrade pytube

In [ ]:
# Librería que permite
# la descarga de videos y audios de múltiples plataformas
pip install yt-dlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.8 MB/s eta 0:00:00


In [ ]:
# Instalación de bibliotecas Python 
# Procesamiento de audio, reducción de ruido y manipulación de archivos sonoros
pip install noisereduce scipy pydub soundfile numpy

In [ ]:
#Importación de bibliotecas
import pytube
import os
import numpy as np
import soundfile as sf
import noisereduce as nr
from scipy.io import wavfile
from pydub import AudioSegment
import re

#### Definición de funciones


Función de descarga del audio de las entrevistas

In [ ]:
#Función utilizada para la descarga del audio de las entrevistas de Youtube
def descargar_audio_youtube(url, output_file="audio.mp4"):
    """
    Descarga solo el audio de un video de YouTube en formato MP4
     Parámetros:
    -----------
    - url (str): URL del video de YouTube del cual se desea extraer el audio.
    - output_file (str, opcional): Nombre del archivo de salida (por defecto "audio.mp4").

    Retorna:
    --------
    - str: Nombre del archivo de audio descargado si la operación es exitosa.
    - None: En caso de error, devuelve None e imprime el mensaje de error.
    """
    try:
        #Limpia el nombre del archivo y se elimina caracteres no permitidos
        nombre_limpio = re.sub(r'[<>:"/\\|?*]', '', output_file)
        #Verifica la extención .mp4 en el nombre del archivo
        if not nombre_limpio.lower().endswith('.mp4'):
            nombre_limpio += '.mp4'

        #Comando txt para ejecución
        #Descarga el formato de audio disponible y lo guarda en un archivo
        comando = f'yt-dlp -f "bestaudio" -o "{nombre_limpio}" {url}'
        #Ejecucion del comando
        resultado = os.system(comando)

        if resultado == 0:
            return nombre_limpio
        else:
            raise Exception("Error en la descarga del audio")

    except Exception as e:
        print(f"Error en la descarga: {str(e)}")
        return None

Función de conversión de audio a formato WAV

In [ ]:
def convertir_a_wav(input_file, output_file="temp.wav"):
    """Convierte el archivo de audio a formato WAV
    Parámetros:
    -----------
    - input_file (str): Ruta del archivo de audio de entrada en cualquier formato compatible (MP3, AAC, FLAC, etc.).
    - output_file (str, opcional): Nombre del archivo de salida en formato WAV (por defecto "temp.wav").

    Retorna:
    --------
    - str: Nombre del archivo de salida en formato WAV si la conversión es exitosa.
    - None: En caso de error, devuelve None e imprime el mensaje de error."""
    try:
        #Carga el archivo de audio a memoria
        audio = AudioSegment.from_file(input_file)
        #Convierte y guarda el archivo de audio en formato WAV
        audio.export(output_file, format="wav")
        return output_file
    except Exception as e:
        print(f"Error en la conversión: {str(e)}")
        return None

Función de limpieza para la reducción de ruido en los archivos de audio


In [ ]:
def limpiar_ruido(archivo_entrada, archivo_salida="audio_limpio.wav", reduccion_ruido=0.5):
    """
    Limpia el ruido del archivo de audio
    Parámetros:
    -----------
    - archivo_entrada (str): Ruta del archivo de audio de entrada.
    - archivo_salida (str, opcional): Nombre del archivo de salida con el audio limpio (por defecto "audio_limpio.wav").
    - reduccion_ruido (float, opcional): Nivel de reducción de ruido (valor entre 0 y 1, por defecto 0.5).

    Retorna:
    --------
    - str: Ruta del archivo de audio limpio si el proceso es exitoso.
    - None: En caso de error, devuelve None e imprime el mensaje de error.
    """
    try:
        if not os.path.exists(archivo_entrada):
            raise FileNotFoundError(f"No se encuentra el archivo: {archivo_entrada}")

        # Si el archivo no está en formato WAV, convertirlo
        if not archivo_entrada.endswith('.wav'):
            archivo_wav = convertir_a_wav(archivo_entrada)
            if not archivo_wav:
                raise Exception("Error en la conversión a WAV")
        else:
            archivo_wav = archivo_entrada

        # Cargar el archivo de audio
        rate, data = wavfile.read(archivo_wav)

        # Convertir a float para el procesamiento
        data = data.astype(float)

        # Si el audio es estéreo, convertirlo a mono
        if len(data.shape) > 1:
            data = np.mean(data, axis=1)

        # Aplicar reducción de ruido
        audio_limpio = nr.reduce_noise(
            y=data,
            sr=rate,
            prop_decrease=reduccion_ruido,
            stationary=True
        )

        # Normalizar el audio
        audio_limpio = audio_limpio / np.max(np.abs(audio_limpio))

        # Guardar el resultado
        sf.write(archivo_salida, audio_limpio, rate)

        # Limpiar archivo temporal si se creó
        if archivo_wav != archivo_entrada:
            os.remove(archivo_wav)

        return archivo_salida

    except Exception as e:
        print(f"Error al procesar el audio: {str(e)}")
        return None


Función de descarga de audio del video de YouTube y aplica la reducción de ruido usando las funciones definidas con anterioridad.

In [ ]:
def procesar_audio_youtube(url, nombre_archivo_original="audio.mp4", nombre_archivo_limpio="audio_limpio.wav"):
    """
    Descarga y procesa un audio de YouTube

    Parámetros:
    - url: URL del video de YouTube
    - nombre_archivo_original: Nombre para guardar el archivo descargado
    - nombre_archivo_limpio: Nombre para guardar el archivo procesado

    Retorna:
    - Ruta del archivo de audio limpio o None si hay error
    """
    # Descargar audio
    archivo_original = descargar_audio_youtube(url, nombre_archivo_original)
    if not archivo_original:
        print("Error: No se pudo descargar el audio")
        return None

    print(f"Audio descargado como: {archivo_original}")

    # Verificar que el archivo existe
    if not os.path.exists(archivo_original):
        print(f"Error: No se encuentra el archivo {archivo_original}")
        return None

    # Limpiar el audio
    archivo_limpio = limpiar_ruido(archivo_original, nombre_archivo_limpio, reduccion_ruido=0.5)

    if archivo_limpio:
        print(f"Audio procesado guardado en: {archivo_limpio}")
        # Opcionalmente, eliminar el archivo original
        # os.remove(archivo_original)
    else:
        print("Error al procesar el audio")

    return archivo_limpio

Importación de la biblioteca Whisper, se usa el modelo preentrenado "small" que ofrece mejor presición con tiempo de procesamiento no tan grandes.

In [ ]:
import whisper

# Cargar el modelo (puede ser 'tiny', 'base', 'small', 'medium' o 'large')
modelo = whisper.load_model("small")

100%|███████████████████████████████████████| 461M/461M [00:12<00:00, 37.6MiB/s]
/usr/local/lib/python3.11/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

Función que mediante el uso de Whisper transcribe los archivos de audio y los guarda en un archivo con extenció .txt

In [ ]:
def transcribir_audio(archivo_limpio="audio_limpio.wav", archivo_transcripcion="transcripcion.txt"):
    """
    Transcribe un archivo de audio con Whisper y guarda el resultado

    Parámetros:
    - archivo_limpio: Ruta del archivo de audio limpio a transcribir
    - archivo_transcripcion: Nombre del archivo donde se guardará la transcripción

    Retorna:
    - El texto transcrito
    """
    try:
        print(f"Iniciando transcripción del archivo: {archivo_limpio}")
        resultado = modelo.transcribe(archivo_limpio)
        texto_transcrito = resultado["text"]

        # Guardar la transcripción en un archivo de texto
        with open(archivo_transcripcion, "w", encoding="utf-8") as f:
            f.write(texto_transcrito)

        print(f"Transcripción guardada en: {archivo_transcripcion}")
        print("\nPrimeras líneas de la transcripción:")
        print("-" * 50)
        print(texto_transcrito[:500] + "...")  # Mostrar las primeras 500 caracteres

        return texto_transcrito

    except Exception as e:
        print(f"Error durante la transcripción: {str(e)}")
        return None


#### Conversión de las entrevistas por candidato

#### Henry Cucalon

Entrevista #1


In [ ]:
#Obtención y procesamiento del audio
audio = procesar_audio_youtube("https://www.youtube.com/watch?v=yH6iLzk9QmE&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=1", "entrevista_Henry_Cucalon_Teleamazonas.mp4", "entrevista_limpia_Henry_Cucalon_Teleamazonas.wav")

Audio descargado como: entrevista_Henry_Cucalon_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Henry_Cucalon_Teleamazonas.wav


In [ ]:
# Transcripción a texto
texto = transcribir_audio("entrevista_limpia_Henry_Cucalon_Teleamazonas.wav", "transcripcion_entrevista_Henry_Cucalon_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Henry_Cucalon_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Henry_Cucalon_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Le doy la bienvenida formalmente. Gracias por acompañarnos esta mañana. Buenos días. Un gusto. Buenos días, buenos días con todos los televidentes. A propósito de varios sucesos, quiero empezar, quizás, conociendo su opinión sobre temas de política exterior. Empezar por estas acciones y decisiones de Donald Trump, con lo ocurrido este impase diplomático con Colombia, que además se convierte como una advertencia, ¿no? Al resto de países que no quieran ajustarse a las políticas del nuevo gobierno...


Entrevista #2 

In [ ]:
audio0 = procesar_audio_youtube("https://www.youtube.com/watch?v=T2qCXLaqJ9c&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=1", "entrevista_Henry_Cucalon_Ecuavisa.mp4", "entrevista_limpia_Henry_Cucalon_Ecuavisa.wav")

Audio descargado como: entrevista_Henry_Cucalon_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Henry_Cucalon_Ecuavisa.wav


In [ ]:
texto0 = transcribir_audio("entrevista_limpia_Henry_Cucalon_Ecuavisa.wav", "transcripcion_entrevista_Henry_Cucalon_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Henry_Cucalon_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Henry_Cucalon_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Hoy cerramos el especial elecciones 2025 los presidenciables con el último candidato que aparecerá en la papeleta Henry Cucalón, cuyo biromio se completa con Carla Larrea. La vida política del guayaquileño de 51 años estuvo ligada en gran parte al partido social cristiano. Luego fue funcionario del gobierno de Creo y ahora lo impulsa, construye Asia Carondellet. En su experiencia destacan cuatro años en la Procuraduría General del Estado de 2003 a 2012 estuvo en el municipio de Guayaquil junto ...


Entrevista #3

In [ ]:
audio1 = procesar_audio_youtube("https://www.youtube.com/watch?v=gpn9gksBsuU&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=1", "entrevista_Henry_Cucalon_RevistaVistazo.mp4", "entrevista_limpia_Henry_Cucalon_RevistaVistazo.wav")

Audio descargado como: entrevista_Henry_Cucalon_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Henry_Cucalon_RevistaVistazo.wav


In [ ]:
texto1 = transcribir_audio("entrevista_limpia_Henry_Cucalon_RevistaVistazo.wav", "transcripcion_entrevista_Henry_Cucalon_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Henry_Cucalon_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Henry_Cucalon_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 He considerado que uno no puede construir su propio futuro si no ayuda a otros a construir el suyo. Tengo una vocación por el Servicio Público, he hecho una carrera en el Servicio Público en la política peldaño a peldaño y considero que desde la Presidencia de la República, el primer servidor de la nación es el espacio de acuerdo ahora para poder ayudar a cambiar los destinos de la nación. Por eso quiero ser presidente del acuerdo. ¿Usted ha hecho una carrera política peldaño a peldaño? Pero la...


#### Ivan Saquicela

Entrevista #1

In [ ]:
audio2 = procesar_audio_youtube("https://www.youtube.com/watch?v=rVy8BAA-Zvw&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=8", "entrevista_Ivan_Saquicela_Teleamazonas.mp4", "entrevista_limpia_Ivan_Saquicela_Teleamazonas.wav")

Audio descargado como: entrevista_Ivan_Saquicela_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Ivan_Saquicela_Teleamazonas.wav


In [ ]:
texto2 = transcribir_audio("entrevista_limpia_Ivan_Saquicela_Teleamazonas.wav", "transcripcion_entrevista_Ivan_Saquicela_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Ivan_Saquicela_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Ivan_Saquicela_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 doctor Zagaysela, gracias por acompañarnos, presentamos una encuesta hace un momento. Prácticamente está todo repartido, nadie más tiene chance de unos que pase una catombe, que Luisa González o Daniel Loboa, Daniel Loboa y Luisa González. Buenos días Milton, gracias por el diálogo. Bueno, yo lo que creo es que está por vez. Hay dos datos muy interesantes. La gran mayoría de los ecuatorianos está pendiente del debate, lo cual los complace porque significa que la gente quiere escuchar la poción,...


Entrevista #2

In [ ]:
audio3 = procesar_audio_youtube("https://www.youtube.com/watch?v=oCDMtxkYRPo&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=4", "entrevista_Ivan_Saquicela_Ecuavisa.mp4", "entrevista_limpia_Ivan_Saquicela_Ecuavisa.wav")

Audio descargado como: entrevista_Ivan_Saquicela_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Ivan_Saquicela_Ecuavisa.wav


In [ ]:
texto3 = transcribir_audio("entrevista_limpia_Ivan_Saquicela_Ecuavisa.wav", "transcripcion_entrevista_Ivan_Saquicela_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Ivan_Saquicela_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Ivan_Saquicela_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Ahora una nueva entrevista dentro de la Especial Elecciones 2025 Los Presidenciables. Iván Saquicela conforma el binomio de democracia así, junto a María Luisa Cuello para participar en los comicios. Saquicela renunció hace cinco meses a la Corte Nacional de Justicia, organismo que también presidió. Fue consejal de cuenca entre 2013 y 2007, luego fue fiscal hasta el año 2015. Además, es docente, universitario, abogado y criminólogo. En el servicio de rentas internas, suma el pago de 46.700 dóla...


#### Francesco Tabacchi

Entrevista #1

In [ ]:
audio4 = procesar_audio_youtube("https://www.youtube.com/watch?v=Yw9PoQGG1TU&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=4", "entrevista_Francesco_Tabacchi_Teleamazonas.mp4", "entrevista_limpia_Francesco_Tabacchi_Teleamazonas.wav")

Audio descargado como: entrevista_Francesco_Tabacchi_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Francesco_Tabacchi_Teleamazonas.wav


In [ ]:
texto4 = transcribir_audio("entrevista_limpia_Francesco_Tabacchi_Teleamazonas.wav", "transcripcion_entrevista_Francesco_Tabacchi_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Francesco_Tabacchi_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Francesco_Tabacchi_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Leo y la bienvenida formalmente a Francesco. Taba que gracias por estar aquí esta mañana. Buenos días. Buenos días. Gracias a usted por la invitación. Por ahí quiero empezar elema de su campaño Mano Dura, Mano Justa, Mano Inteligente. ¿Para qué? ¿Para quién es? Exactamente. Mano Dura para los delincuentes. Mano Dura para los terroristas. Mano Dura para los jueces y fiscales corruptos. Una mano justa para todas las personas del ecuador que necesitan oportunidades. Hasta para los hijos de los del...


Entrevista #2

In [ ]:
audio5 = procesar_audio_youtube("https://www.youtube.com/watch?v=PzsZxCG4Eds&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=11", "entrevista_Francesco_Tabacchi_Ecuavisa.mp4", "entrevista_limpia_Francesco_Tabacchi_Ecuavisa.wav")

Audio descargado como: entrevista_Francesco_Tabacchi_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Francesco_Tabacchi_Ecuavisa.wav


In [ ]:
texto5 = transcribir_audio("entrevista_limpia_Francesco_Tabacchi_Ecuavisa.wav", "transcripcion_entrevista_Francesco_Tabacchi_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Francesco_Tabacchi_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Francesco_Tabacchi_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 en el especial elecciones, avanzamos en el diálogo con los candidatos, hoy está en contacto directo Francesco Tabaqui, el ganadero de 53 años, quien conforma con Blanca Saquicela, el binomio del movimiento Creo, Blanca Saquicela, Blanca Saquicela, su experiencia política, la de Francesco Tabaqui y Gremial, se resume a ser gobernador del guayas en 2023 y candidato a prefecto de guayas con la misma agrupación, además fue presidente de la asociación de ganaderos de litoral y Galápagos también enca...


Entrevista #3

In [ ]:
audio6 = procesar_audio_youtube("https://www.youtube.com/watch?v=C85olI5o3RA&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76", "entrevista_Francesco_Tabacchi_RevistaVistazo.mp4", "entrevista_limpia_Francesco_Tabacchi_RevistaVistazo.wav")

Audio descargado como: entrevista_Francesco_Tabacchi_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Francesco_Tabacchi_RevistaVistazo.wav


In [ ]:
texto6 = transcribir_audio("entrevista_limpia_Francesco_Tabacchi_RevistaVistazo.wav", "transcripcion_entrevista_Francesco_Tabacchi_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Francesco_Tabacchi_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Francesco_Tabacchi_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 ¿Qué es lo que es la historia del país? ¿Qué es el país del país del país? ¿Qué es el país del país del país del país del país? ¿Qué es el país del país del país del país del país? La misma pregunta me hace mi familia, mi esposa, mis hijos, mis padres, me hacen mis socios. Y este país ha sido muy generoso conmigo. Me he caído, me he levantado. Pero, afinitivamente, este país ha sido bueno conmigo. Víe el momento y la forma de retribuírselo. Creo en mi país, creo en el Ecuador. Porque ahora, si ...


#### Andrea Gonzalez

Entrevista #1

In [ ]:
audio7 = procesar_audio_youtube("https://www.youtube.com/watch?v=z-YfW2WLNeY&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=9", "entrevista_Andrea_Gonzalez_Teleamazonas.mp4", "entrevista_limpia_Andrea_Gonzalez_Teleamazonas.wav")

Audio descargado como: entrevista_Andrea_Gonzalez_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Andrea_Gonzalez_Teleamazonas.wav


In [ ]:
texto7 = transcribir_audio("entrevista_limpia_Andrea_Gonzalez_Teleamazonas.wav", "transcripcion_entrevista_Andrea_Gonzalez_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Andrea_Gonzalez_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Andrea_Gonzalez_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Y ya está con nosotros como lo ven aquí en el estudio en Waiquilandrea González. Bienvenida, gracias por acompañarnos. Muchísimas gracias Liz, me siento como en casa. Ya veíamos su perfil, usted ya es una figura política conocida, pero sin embargo lo que algunos cuestionan es que no ha ejercido dentro de la función pública y sin haberlo hecho, ¿cuál va a ser la experiencia para dirigir un país? Justamente el tema de ser de la clase trabajadora y entender que un gobierno cuando no ayuda estorba ...


Entrevista #2

In [ ]:
audio8 = procesar_audio_youtube("https://www.youtube.com/watch?v=UQJ09FRKAZk&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=12", "entrevista_Andrea_Gonzalez_Ecuavisa.mp4", "entrevista_limpia_Andrea_Gonzalez_Ecuavisa.wav")

Audio descargado como: entrevista_Andrea_Gonzalez_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Andrea_Gonzalez_Ecuavisa.wav


In [ ]:
texto8 = transcribir_audio("entrevista_limpia_Andrea_Gonzalez_Ecuavisa.wav", "transcripcion_entrevista_Andrea_Gonzalez_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Andrea_Gonzalez_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Andrea_Gonzalez_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 En la especial elección es 2025 los presidenciables llegó el turno de conversar con Andrea González Nader, la guayaquileña de 37 años quien junto a Galo Munkayo conforman el binomio presidencial de Sociedad Patriótica. En la campaña anterior Andrea González fue la carta a la vicepresidencia de Construye con el activista Fernando Villavicencio quien fuera asesinado el 9 de agosto de 2023 pero no han sido esas sus únicas camisetas políticas. En 2019 patrocinada por unidad popular quiso ser vicepr...


Entrevista #3

In [ ]:
audio9 = procesar_audio_youtube("https://www.youtube.com/watch?v=b9DmUReNKNg&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=10", "entrevista_Andrea_Gonzalez_RevistaVistazo.mp4", "entrevista_limpia_Andrea_Gonzalez_RevistaVistazo.wav")

Audio descargado como: entrevista_Andrea_Gonzalez_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Andrea_Gonzalez_RevistaVistazo.wav


In [ ]:
texto9 = transcribir_audio("entrevista_limpia_Andrea_Gonzalez_RevistaVistazo.wav", "transcripcion_entrevista_Andrea_Gonzalez_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Andrea_Gonzalez_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Andrea_Gonzalez_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 ¿Por qué estos candidatos están en estos momentos participando? En mi caso es una continuidad de lo que fue el año pasado, no solamente de recorrer todo el territorio y aterrizar un plan de gobierno al que me dejaron construir como candidata a la vicepresidencia. Yo vengo de una vicepresidencia muy empoderada del año pasado con mi binomio y pude participar activamente en la construcción de ese plan de gobierno. Este año la responsabilidad fue netamente mía de construir este plan de gobierno que...


#### Juan Ivan Cueva

Entrevista #1

In [ ]:
audio10 = procesar_audio_youtube("https://www.youtube.com/watch?v=aC08x8rQ8Ts&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=15", "entrevista_Juan_Ivan_Cueva_Teleamazonas.mp4", "entrevista_limpia_Juan_Ivan_Cueva_Teleamazonas.wav")

Audio descargado como: entrevista_Juan_Ivan_Cueva_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Juan_Ivan_Cueva_Teleamazonas.wav


In [ ]:
texto10 = transcribir_audio("entrevista_limpia_Juan_Ivan_Cueva_Teleamazonas.wav", "transcripcion_entrevista_Juan_Ivan_Cueva_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Juan_Ivan_Cueva_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Juan_Ivan_Cueva_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 pero Juan Iván Cueva es el primer candidato que va a estar con nosotros, es el candidato del movimiento amigo. Veamos un pequeño reportaje de quién es el candidato presidencial de amigo. Con cerca de 10 años de experiencia como empresario pero con mucho menos, como político, Juan Iván Cueva Vivanco nacido en Lojas pira llegar al Palacio de Carondelet. Es ingeniero en electrónica y telecomunicaciones, rama en la que mejor se desenvolve según su perfil personal en el cual se detalla que es especi...


Entrevista #2

In [ ]:
audio11 = procesar_audio_youtube("https://www.youtube.com/watch?v=439W0-4DLlU&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=3", "entrevista_Juan_Ivan_Cueva_Ecuavisa.mp4", "entrevista_limpia_Juan_Ivan_Cueva_Ecuavisa.wav")

Audio descargado como: entrevista_Juan_Ivan_Cueva_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Juan_Ivan_Cueva_Ecuavisa.wav


In [ ]:
texto11 = transcribir_audio("entrevista_limpia_Juan_Ivan_Cueva_Ecuavisa.wav", "transcripcion_entrevista_Juan_Ivan_Cueva_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Juan_Ivan_Cueva_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Juan_Ivan_Cueva_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 En el especial de lecciones 2025 los presidenciables hoy estará aquí Juan Iván Cueva candidato por el movimiento Amigo en binomio con Cristina Reyes tiene experiencia en la función pública desde 2018 a 2019 en medios públicos luego en la corporación de telecomunicaciones y el año pasado estuvo en el ministerio de telecomunicaciones su patrimonio es de 526.295 dólares y figura como presidente de dos compañías y gerente general de una tercera por impuesto a la renta apagado 7.700 dólares desde 20...


Entrevista #3


In [ ]:
audio12 = procesar_audio_youtube("https://www.youtube.com/watch?v=eoNipPmug5w&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=5", "entrevista_Juan_Ivan_Cueva_RevistaVistazo.mp4", "entrevista_limpia_Juan_Ivan_Cueva_RevistaVistazo.wav")

Audio descargado como: entrevista_Juan_Ivan_Cueva_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Juan_Ivan_Cueva_RevistaVistazo.wav


In [ ]:
texto12 = transcribir_audio("entrevista_limpia_Juan_Ivan_Cueva_RevistaVistazo.wav", "transcripcion_entrevista_Juan_Ivan_Cueva_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Juan_Ivan_Cueva_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Juan_Ivan_Cueva_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 ¿Por qué soy un ciudadano al igual que la mayoría de cuatorianos, de los 18 millones de cuatorianos? Yo soy uno más de la ciudadanía que se ha preparado durante muchísimos años para correr por esta candidatura a la Presidencia para correr hacia el Palacio de Perón del Ed, lograr ese cambio profundo que necesitamos porque estoy aquí para defender al país de los corruptos del pasado y de los ineptos que hoy nos están gobernando. He decidido dar un paso hacia adelante y no dar un paso hacia el cos...


#### Carlos Rabascall

Entrevista #1

In [ ]:
audio13 = procesar_audio_youtube("https://www.youtube.com/watch?v=tKOrCZnUI8A&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=6", "entrevista_Carlos_Rabascall_Teleamazonas.mp4", "entrevista_limpia_Carlos_Rabascall_Teleamazonas.wav")

Audio descargado como: entrevista_Carlos_Rabascall_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Carlos_Rabascall_Teleamazonas.wav


In [ ]:
texto13 = transcribir_audio("entrevista_limpia_Carlos_Rabascall_Teleamazonas.wav", "transcripcion_entrevista_Carlos_Rabascall_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Carlos_Rabascall_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Carlos_Rabascall_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Le quiero dar la bienvenida formalmente al ingeniero Carlos Rabascar. Gracias por estar con nosotros esta mañana. Buenos días. Muy buenos días, Luis. Que gusto verte y felicitaciones por el estudio. Está precioso. Muchísimas gracias. No quiero engancharme mucho en este tema, pero no quiero tampoco dejar de preguntarle... La edad ya la dije. No, ya está. Más bien, su percepción del debate del domingo. A ver, yo creo que todo espacio es importante, por más que el formato es una camisa de fuerza c...


Entrevista #2


In [ ]:
audio14 = procesar_audio_youtube("https://www.youtube.com/watch?v=wUU6DbSs-7s&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=7", "entrevista_Carlos_Rabascall_Ecuavisa.mp4", "entrevista_limpia_Carlos_Rabascall_Ecuavisa.wav")

Audio descargado como: entrevista_Carlos_Rabascall_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Carlos_Rabascall_Ecuavisa.wav


In [ ]:
texto14 = transcribir_audio("entrevista_limpia_Carlos_Rabascall_Ecuavisa.wav", "transcripcion_entrevista_Carlos_Rabascall_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Carlos_Rabascall_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Carlos_Rabascall_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 en el especial elecciones dos mil veinticinco los presidenciables hoy la entrevista será a Carlos Ravascal quien conforme el binomio de izquierda democrática junto con Alejandra Rivas el ingeniero comercial de 64 años en 2021 fue candidato a la vicepresidencia por la alianza centro democrático fuerza compromiso social que apoyó al correismo con Andrés Arausa la cabeza el año pasado Ravascal se afilió a la izquierda democrática el fue presentador y entrevistador en ecuador tv entre 2004 y 2017 t...


Entrevista #3

In [ ]:
audio15 = procesar_audio_youtube("https://www.youtube.com/watch?v=d52jHpgBgOk&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=6", "entrevista_Carlos_Rabascall_RevistaVistazo.mp4", "entrevista_limpia_Carlos_Rabascall_RevistaVistazo.wav")

Audio descargado como: entrevista_Carlos_Rabascall_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Carlos_Rabascall_RevistaVistazo.wav


In [ ]:
texto15 = transcribir_audio("entrevista_limpia_Carlos_Rabascall_RevistaVistazo.wav", "transcripcion_entrevista_Carlos_Rabascall_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Carlos_Rabascall_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Carlos_Rabascall_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 ¿Por qué quiero ser candidato? Porque soy muy malo para quedarme paralizado y para solamente ver cómo el país está destruyendo. Hace 45 años regresamos a la democracia ecotoriana luego de una dictadura y seguimos teniendo los mismos problemas, problemas estructurales, pero más agravado, más intensificado, débil estructura económica, débil estructura de las finanzas públicas, incapacidad para generar trabajo pleno, adecuado, formal, modelo educativo que no ha ido con la evolución del mundo conte...


#### Luis Felipe Tilleria

Entrevista #1


In [ ]:
audio16 = procesar_audio_youtube("https://www.youtube.com/watch?v=mde_6gkkrek&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=14", "entrevista_Luis_Felipe_Tilleria_Teleamazonas.mp4", "entrevista_limpia_Luis_Felipe_Tilleria_Teleamazonas.wav")

Audio descargado como: entrevista_Luis_Felipe_Tilleria_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Luis_Felipe_Tilleria_Teleamazonas.wav


In [ ]:
texto16 = transcribir_audio("entrevista_limpia_Luis_Felipe_Tilleria_Teleamazonas.wav", "transcripcion_entrevista_Luis_Felipe_Tilleria_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Luis_Felipe_Tilleria_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Luis_Felipe_Tilleria_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Y le doy ya la bienvenida formalmente al Ifeleluipete Yería, candidato a la Presidencia por Avanza, que gusta tenerlo aquí. Buenos días, bienvenido. Sí, buenos días. Antes que nada quisiera entregarle mi currículum. Vengo a una entrevista de trabajo. Aspiro a tener el trabajo de presidente de 18 millones de cuantos años. Y usted migró en el año 2000, primer Estados Unidos, 2002. Más bien ya se va pues a Inglaterra, en Londres, donde ha permanecido durante todos estos años. Y eso no lo ponen de ...


Entrevista #2

In [ ]:
audio17 = procesar_audio_youtube("https://www.youtube.com/watch?v=5oADrbQ5Q10&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=8", "entrevista_Luis_Felipe_Tilleria_Ecuavisa.mp4", "entrevista_limpia_Luis_Felipe_Tilleria_Ecuavisa.wav")

Audio descargado como: entrevista_Luis_Felipe_Tilleria_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Luis_Felipe_Tilleria_Ecuavisa.wav


In [ ]:
texto17 = transcribir_audio("entrevista_limpia_Luis_Felipe_Tilleria_Ecuavisa.wav", "transcripcion_entrevista_Luis_Felipe_Tilleria_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Luis_Felipe_Tilleria_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Luis_Felipe_Tilleria_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 y continuamos con el especial elecciones, dos mil veinticinco, los presidenciables para conversar hoy con Luis Felipe Tillería, quien junto a Carla Rosero conforman el binomio de avanza. El Boyaquileño de 42 años es nuevo en política local, pero figura como consejal en uno de los distritos de Londres, país al que migró en dos mil tres, y ese cargo, el de consejal en Londres, lo debe ocupar hasta el mes de marzo. El del latinoamericano en Londres también fue gerente del Royal Bank recipientado d...


Entrevista #3

In [ ]:
audio18 = procesar_audio_youtube("https://www.youtube.com/watch?v=F7TAfbJkaZ8&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=7", "entrevista_Luis_Felipe_Tilleria_RevistaVistazo.mp4", "entrevista_limpia_Luis_Felipe_Tilleria_RevistaVistazo.wav")

Audio descargado como: entrevista_Luis_Felipe_Tilleria_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Luis_Felipe_Tilleria_RevistaVistazo.wav


In [ ]:
texto18 = transcribir_audio("entrevista_limpia_Luis_Felipe_Tilleria_RevistaVistazo.wav", "transcripcion_entrevista_Luis_Felipe_Tilleria_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Luis_Felipe_Tilleria_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Luis_Felipe_Tilleria_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 ¿Por qué el pueblo de México está en el país más rico de Latinoamérica? ¿Por qué el pueblo de México está en el país más rico de Latinoamérica? ¿Por qué el pueblo de México está en el país más rico de Latinoamérica? Porque siempre he tenido la certeza que Ecuador puede ser un país del primer mundo. Ecuador está destinado a ser el país más rico de Latinoamérica. Y esto sí se puede hacer en una generación. Mi papá es chileno y mi papá vino a Ecuador en busca de oportunidades hace 48 años. Porque ...


#### Henry Kronfle


Entrevista #1

In [ ]:
audio19 = procesar_audio_youtube("https://www.youtube.com/watch?v=khfD1sTj73A&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=3", "entrevista_Henry_Kronfle_Teleamazonas.mp4", "entrevista_limpia_Henry_Kronfle_Teleamazonas.wav")

Audio descargado como: entrevista_Henry_Kronfle_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Henry_Kronfle_Teleamazonas.wav


In [ ]:
texto19 = transcribir_audio("entrevista_limpia_Henry_Kronfle_Teleamazonas.wav", "transcripcion_entrevista_Henry_Kronfle_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Henry_Kronfle_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Henry_Kronfle_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 ¡Genericron! ¡Gracias por acompañar los buenos días! Gracias por la invitación, Milton. Antes de conocer algunas propuestas, aclaremos algo. Usted dijo, voy a extraditar a Rafael Correa. ¿Cómo andará a traer? Bueno, existe un contrato de un convenio de extradición con el Reino de Bélgica, que data de finales del siglo XIX, primero. Segundo, los delitos tipificados en el ecuador también existen allá, que eso es una condición sin ecuadrón. Tercero, se han cometido delitos, no se puede hablar que ...


Entrevista #2

In [ ]:
audio20 = procesar_audio_youtube("https://www.youtube.com/watch?v=yosPW9f7-4A&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=9", "entrevista_Henry_Kronfle_Ecuavisa.mp4", "entrevista_limpia_Henry_Kronfle_Ecuavisa.wav")

Audio descargado como: entrevista_Henry_Kronfle_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Henry_Kronfle_Ecuavisa.wav


In [ ]:
texto20 = transcribir_audio("entrevista_limpia_Henry_Kronfle_Ecuavisa.wav", "transcripcion_entrevista_Henry_Kronfle_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Henry_Kronfle_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Henry_Kronfle_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 la semana avanzamos en la papeleta de las elecciones 2025 para hablar con los presidenciables. Hoy con Henry Cronefle quien encabeza el binomio social cristiano que conforma con Dayana Pasleg. El candidato de la CES acumuló tres periodos consecutivos como asambleista nacional y ocupó la presidencia de este organismo en el último de sus periodos. Antes fue Consul Honorario de México y como dirigente gremial, presidente de la Asociación de la CES, el agosto de la presidencia de la CES. Según la C...


Entrevista #3

In [ ]:
audio21 = procesar_audio_youtube("https://www.youtube.com/watch?v=rnl4dXWjy8U&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=8", "entrevista_Henry_Kronfle_RevistaVistazo.mp4", "entrevista_limpia_Henry_Kronfle_RevistaVistazo.wav")

Audio descargado como: entrevista_Henry_Kronfle_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Henry_Kronfle_RevistaVistazo.wav


In [ ]:
texto21 = transcribir_audio("entrevista_limpia_Henry_Kronfle_RevistaVistazo.wav", "transcripcion_entrevista_Henry_Kronfle_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Henry_Kronfle_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Henry_Kronfle_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Quiero ser candidato a la Presidencia porque amo este país y veo que cada día se profundizan más los problemas que son graves y urgentes, sobre todo los que tienen la población en general y la gente más pobre. Cuando estuve en la Asamblea Nacional muchos años y en el último año como presidente de la Asamblea Nacional, atendía mucha gente, recibía decenas de miles de personas durante casi ocho años en la Asamblea Nacional y escuchando todo tipo de representantes de ganaderos, agricultores, pesca...


#### Victor Araus

Entrevista #1

In [ ]:
audio22 = procesar_audio_youtube("https://www.youtube.com/watch?v=1-pCYCFfcts&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=13", "entrevista_Victor_Araus_Teleamazonas.mp4", "entrevista_limpia_Victor_Araus_Teleamazonas.wav")

Audio descargado como: entrevista_Victor_Araus_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Victor_Araus_Teleamazonas.wav


In [ ]:
texto22 = transcribir_audio("entrevista_limpia_Victor_Araus_Teleamazonas.wav", "transcripcion_entrevista_Victor_Araus_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Victor_Araus_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Victor_Araus_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 de la ciudad de México. Gracias por acompañarnos. Gracias a ustedes por la invitación. Un saludo a todo el país. Quiero empezar por esto último que decía el reportaje. Usted vio una entrevista suya de hace par de meses atrás en la posta. Decía que el embajador de los Estados Unidos, Michael Fitzpatrick, lo persiguió y usted lo acusó de corrupto. Aclaren esto para empezar. 100 por ciento Milton porque el embajador en primer lugar se convirtió en un cómple silencioso y después fue parte important...


Entrevista #2

In [ ]:
audio23 = procesar_audio_youtube("https://www.youtube.com/watch?v=xxdeZPBY_Jc&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=9", "entrevista_Victor_Araus_RevistaVistazo.mp4", "entrevista_limpia_Victor_Araus_RevistaVistazo.wav")

Audio descargado como: entrevista_Victor_Araus_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Victor_Araus_RevistaVistazo.wav


In [ ]:
texto23 = transcribir_audio("entrevista_limpia_Victor_Araus_RevistaVistazo.wav", "transcripcion_entrevista_Victor_Araus_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Victor_Araus_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Victor_Araus_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 36 años de servicio de la Policía Nacional no ha permitido conocer mucho la realidad de nuestro país. Conozco todos los ricones de nuestra patria y conozco al detalle cada una de sus preocupaciones, cada una de sus inquietudes y la actual situación en la que vi el país que estamos rodillados ante la delincuencia, una crisis económica terrible, una crisis energética que nos afectaba a todos más que incluso la misma pandemia y con ese conocimiento y solvencia profesional como digo yo. Creo que te...


#### Jorge Escala

Entrevista #1

In [ ]:
audio24 = procesar_audio_youtube("https://www.youtube.com/watch?v=oeALtyL5joA&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=10", "entrevista_Jorge_Escala_Teleamazonas.mp4", "entrevista_limpia_Jorge_Escala_Teleamazonas.wav")

Audio descargado como: entrevista_Jorge_Escala_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Jorge_Escala_Teleamazonas.wav


In [ ]:
texto24 = transcribir_audio("entrevista_limpia_Jorge_Escala_Teleamazonas.wav", "transcripcion_entrevista_Jorge_Escala_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Jorge_Escala_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Jorge_Escala_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Y lo doy formalmente la bienvenida y las gracias por acompañarnos a Jorge Scala, candidato por un día popular a la presidencia de la República. Buenos días. Buenos días, Liz. ¿Qué gusto estar con usted a los tiempos? Bueno, a los tiempos, es verdad. Después de 12 años, prácticamente usted regresa a este trajín político. Bueno, no me ha ausentado de la lucha social y popular. Cierto es que he estado dedicado a mi labor docente. Son 32 años dedicado a la formación de la niñez y la juventud. La ge...


Entrevista #2

In [ ]:
audio25 = procesar_audio_youtube("https://www.youtube.com/watch?v=xHrZg7A65bQ&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=13", "entrevista_Jorge_Escala_Ecuavisa.mp4", "entrevista_limpia_Jorge_Escala_Ecuavisa.wav")

Audio descargado como: entrevista_Jorge_Escala_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Jorge_Escala_Ecuavisa.wav


In [ ]:
texto25 = transcribir_audio("entrevista_limpia_Jorge_Escala_Ecuavisa.wav", "transcripcion_entrevista_Jorge_Escala_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Jorge_Escala_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Jorge_Escala_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 y seguimos con el especial de lecciones, dos mil veinticinco los presidenciables. Hoy vino a contacto directo a Jorge Escala, quien mañana va a cumplir cincuenta y cinco años y en cabeza el binomio de unidad popular, binomio que lo completa Pacha Terán. Toda su carrera política ha sido en el mismo partido que antes se llamaba MPD, el movimiento popular democrático y es unidad popular desde hace algunos años. Desde mil novecientos noventa y tres años, la Comisión Legislativa y de Fiscalización r...


Entrevista #3

In [ ]:
audio26 = procesar_audio_youtube("https://www.youtube.com/watch?v=GTgeoo62qPs&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=11", "entrevista_Jorge_Escala_RevistaVistazo.mp4", "entrevista_limpia_Jorge_Escala_RevistaVistazo.wav")

Audio descargado como: entrevista_Jorge_Escala_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Jorge_Escala_RevistaVistazo.wav


In [ ]:
texto26 = transcribir_audio("entrevista_limpia_Jorge_Escala_RevistaVistazo.wav", "transcripcion_entrevista_Jorge_Escala_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Jorge_Escala_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Jorge_Escala_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 ¿Por qué los niños y los jóvenes tienen el derecho humano a la educación frente a lo que está ocurriendo el día de hoy, alrededor de 200.000 niños fuera de sus aulas, estamos hablando casi un millón de jóvenes, bachilleres en acceso a la universidad, porque quiero que la gente tenga salud, hay que proteger la vida y la gente no puede seguir muriendo en los hospitales, porque quiero que se reactive la producción, ya basta que nuestros agricultores y ganaderos estén abandonados y quebrados, porqu...


#### Jimmy Jairala

Entrevista #1

In [ ]:
audio27 = procesar_audio_youtube("https://www.youtube.com/watch?v=k4xb-HMOcZE&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=5", "entrevista_Jimmy_Jairala_Teleamazonas.mp4", "entrevista_limpia_Jimmy_Jairala_Teleamazonas.wav")

Audio descargado como: entrevista_Jimmy_Jairala_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Jimmy_Jairala_Teleamazonas.wav


In [ ]:
texto27 = transcribir_audio("entrevista_limpia_Jimmy_Jairala_Teleamazonas.wav", "transcripcion_entrevista_Jimmy_Jairala_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Jimmy_Jairala_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Jimmy_Jairala_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 que nos ha dado un saludo a Dimitri Gala gracias para acompañarnos. No ha pagado impuesto a la renta. Me dame atención. Asecuatro años. Asecuatro años. Soy jubilado. Tenemos un régimen especial, pero mi empresa, que se menciona en este reportaje, ha pagado $35,000 en los últimos años. Se llama SEOS, justamente la que refería el reportaje. No hay absolutamente ningún pendiente con el tema de la contraloría ni con la fiscalía. Fíjese usted cómo es la vida. Los dos contralores que me denunciaron y...


Entrevista #2

In [ ]:
audio28 = procesar_audio_youtube("https://www.youtube.com/watch?v=DE_-9EbjR_Y&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=5", "entrevista_Jimmy_Jairala_Ecuavisa.mp4", "entrevista_limpia_Jimmy_Jairala_Ecuavisa.wav")

Audio descargado como: entrevista_Jimmy_Jairala_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Jimmy_Jairala_Ecuavisa.wav


In [ ]:
texto28 = transcribir_audio("entrevista_limpia_Jimmy_Jairala_Ecuavisa.wav", "transcripcion_entrevista_Jimmy_Jairala_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Jimmy_Jairala_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Jimmy_Jairala_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 En la especial elecciones 2025 los presidenciables. Hoy entrevistaremos a Jimmy Jairala, quien completó el binomio de Centro Democrático con Lucía Vallesilla, ex presidenta del Club del Nacional. Es su primer intento por llegar a Caron de Lett, pero antes fue postulante a otras dignidades con las camisetas de los extintos Partido Roldosiste Cuatoriano y el Movimiento 1. La alcaldía de Guayaquil le fue esquiva en dos ocasiones en 2004 y en 2019. En cambio estuvo en la perfectura del Guayas duran...


Entrevista #3

In [ ]:
audio29 = procesar_audio_youtube("https://www.youtube.com/watch?v=3U4RP1pQSbg&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=12", "entrevista_Jimmy_Jairala_RevistaVistazo.mp4", "entrevista_limpia_Jimmy_Jairala_RevistaVistazo.wav")

Audio descargado como: entrevista_Jimmy_Jairala_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Jimmy_Jairala_RevistaVistazo.wav


In [ ]:
texto29 = transcribir_audio("entrevista_limpia_Jimmy_Jairala_RevistaVistazo.wav", "transcripcion_entrevista_Jimmy_Jairala_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Jimmy_Jairala_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Jimmy_Jairala_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Quiero ser candidato a la presidencia porque soy parte de ese 60% de ciudadanos que están astiados de la polarización, están astiados de la propia política, están decepcionados del cuadro que uno presenta una futura elección. Candidatos que nos han polarizado, que nos han etiquetado, que nos han señalado como buenos, malos, blancos, negros, estoy contigo, estoy contra anticorreista, anticorreista y yo me sumo a ese 60% de ciudadanos que no quieren que los etiqueten. Yo me niego a que me etiquet...


#### Leonidas Iza

Entrevista #1

In [ ]:
audio30 = procesar_audio_youtube("https://www.youtube.com/watch?v=PXEfP9k2YfU&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=2", "entrevista_Leonidas_Iza_Teleamazonas.mp4", "entrevista_limpia_Leonidas_Iza_Teleamazonas.wav")

Audio descargado como: entrevista_Leonidas_Iza_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Leonidas_Iza_Teleamazonas.wav


In [ ]:
texto30 = transcribir_audio("entrevista_limpia_Leonidas_Iza_Teleamazonas.wav", "transcripcion_entrevista_Leonidas_Iza_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Leonidas_Iza_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Leonidas_Iza_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 de la ciudad. Buenos días, Lidia, Disha, gracias por acompañarnos. Quiero empezar con lo que está ocurriendo ahora mismo con la política de Donald Trump y presidente de los Estados Unidos. Lo de Colombia ayer fue una muestra del poder que puede ejercer el presidente de los Estados Unidos en el tema de los migrantes en este momento. Usted gana las elecciones, cuál sería su reacción inmediata para enfrentar ese tema. Buenos días pueblo ecuatoriano, gracias por la invitación. Este es el neoliberal...


Entrevista #2

In [ ]:
audio31 = procesar_audio_youtube("https://www.youtube.com/watch?v=4FLO0BIGl4U&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=3", "entrevista_Leonidas_Iza_Ecuavisa.mp4", "entrevista_limpia_Leonidas_Iza_Ecuavisa.wav")

Audio descargado como: entrevista_Leonidas_Iza_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Leonidas_Iza_Ecuavisa.wav


In [ ]:
texto31 = transcribir_audio("entrevista_limpia_Leonidas_Iza_Ecuavisa.wav", "transcripcion_entrevista_Leonidas_Iza_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Leonidas_Iza_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Leonidas_Iza_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 y los que están en el país. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .izingrate poco de zaman. para el sector de la salud. Con el decreto 456, 454, perdón, el gobierno nacional dispuso 200 millones de dólares en emergencia de salud. Con el decreto 456, el gobierno dispuso 70 millones de dólares para el bono de desarrollo humano. Y con los siguientes ochos decretos, en condonación de deuda, significaron 44 millones de dólares. Entonces mi referencia en ...


#### Pedro Granja

Entrevista #1

In [ ]:
audio32 = procesar_audio_youtube("https://www.youtube.com/watch?v=s7yWMg_s1VI&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=11", "entrevista_Pedro_Granja_Teleamazonas.mp4", "entrevista_limpia_Pedro_Granja_Teleamazonas.wav")

Audio descargado como: entrevista_Pedro_Granja_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Pedro_Granja_Teleamazonas.wav


In [ ]:
texto32 = transcribir_audio("entrevista_limpia_Pedro_Granja_Teleamazonas.wav", "transcripcion_entrevista_Pedro_Granja_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Pedro_Granja_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Pedro_Granja_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Y hasta con nosotros en el estudio. Su abogado, bienvenido. Gracias por acompañarnos, por ampliar sus propuestas de campaña esta mañana. Lindo día, Liz. Tenía muchísimas ganas de conversar con ustedes. Bueno, primero que nada, me pido por preguntarle, ¿no lo ponen desventaja el hecho de que usted no está haciendo campaña en territorio? Sin duda ninguna, pero hacer campaña en territorio con el informe de inteligencia que me sitúa como uno de los candidatos con mayores opciones de ser asesinado p...


Entrevista #2

In [ ]:
audio33 = procesar_audio_youtube("https://www.youtube.com/watch?v=o_Qcvb2LfMQ&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=6", "entrevista_Pedro_Granja_Ecuavisa.mp4", "entrevista_limpia_Pedro_Granja_Ecuavisa.wav")

Audio descargado como: entrevista_Pedro_Granja_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Pedro_Granja_Ecuavisa.wav


In [ ]:
texto33 = transcribir_audio("entrevista_limpia_Pedro_Granja_Ecuavisa.wav", "transcripcion_entrevista_Pedro_Granja_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Pedro_Granja_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Pedro_Granja_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 el presidente de la ciudad. El invitado de hoy en el especial elecciones dos mil veinticinco los presidenciables es Pedro Granja quien conforme el binomio del partido socialista ecuatoriano con Verónica Silva además de jurista y criminólogo fue secretario de la Federación de Abogados del Ecuador durante el correísmo y figura como docente universitario en el pago de impuestos suma tres mil mil millones de pesos de salida de divisas además es presidente de una compañía dedicada a enseñanza, capac...


Entrevista #3

In [ ]:
audio34 = procesar_audio_youtube("https://www.youtube.com/watch?v=klZZVxXrhHM&list=PL4ukCuVPXaeymuyrMV2QFdhBlKRcpNZ76&index=2", "entrevista_Pedro_Granja_RevistaVistazo.mp4", "entrevista_limpia_Pedro_Granja_RevistaVistazo.wav")

Audio descargado como: entrevista_Pedro_Granja_RevistaVistazo.mp4
Audio procesado guardado en: entrevista_limpia_Pedro_Granja_RevistaVistazo.wav


In [ ]:
texto34 = transcribir_audio("entrevista_limpia_Pedro_Granja_RevistaVistazo.wav", "transcripcion_entrevista_Pedro_Granja_RevistaVistazo.txt")

Iniciando transcripción del archivo: entrevista_limpia_Pedro_Granja_RevistaVistazo.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Pedro_Granja_RevistaVistazo.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Música Música Mira, creo que hace 24 años vengo defendiendo niños víctimas de abuso sexual. Estuve conversando muchísimo sobre si decirlo o no, si confesarlo o no. Hace varios años con mi familia, fundamentalmente con mi mujer, con mi compañera. Sobre lo que me ocurrió a mí cuando fui muy pequeño, yo soy un sobreviviente. Creo que he llevado hoyas populares, están los testimonios, no están allí, se puede revisar fácilmente. Hace muchísimos años a los barrios más pobres del Ecuador, brigadas méd...


#### Luisa Gonzalez

Entrevista #1


In [ ]:
audio35 = procesar_audio_youtube("https://www.youtube.com/watch?v=qtv8pyvNx0U&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=7", "entrevista_Luisa_Gonzalez_Teleamazonas.mp4", "entrevista_limpia_Luisa_Gonzalez_Teleamazonas.wav")

Audio descargado como: entrevista_Luisa_Gonzalez_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Luisa_Gonzalez_Teleamazonas.wav


In [ ]:
texto35 = transcribir_audio("entrevista_limpia_Luisa_Gonzalez_Teleamazonas.wav", "transcripcion_entrevista_Luisa_Gonzalez_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Luisa_Gonzalez_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Luisa_Gonzalez_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Muchas gracias, buenos días, gracias por acompañarlos. Quiero empezar por el debate. Un ejercicio largo, democrático, sí. Difícil el formato, porque son tantos candidatos también. Pero cómo se sintió usted, considera que le fue bien, considera que usted se llevó, digamos, el debate ayer. Tiene tres segundos para responder. No me tiras este. Muchas gracias, un buen día con todos los ecuatorianos que nos están mirando en este momento. Creo que aquí quien debe ganar o perder es el pueblo ecuatoria...


Entrevista #2

In [ ]:
audio36 = procesar_audio_youtube("https://www.youtube.com/watch?v=tdYZoUeVuSU&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=10", "entrevista_Luisa_Gonzalez_Ecuavisa.mp4", "entrevista_limpia_Luisa_Gonzalez_Ecuavisa.wav")

Audio descargado como: entrevista_Luisa_Gonzalez_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Luisa_Gonzalez_Ecuavisa.wav


In [ ]:
texto36 = transcribir_audio("entrevista_limpia_Luisa_Gonzalez_Ecuavisa.wav", "transcripcion_entrevista_Luisa_Gonzalez_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Luisa_Gonzalez_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Luisa_Gonzalez_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 En la especial elección es 2025 los presidenciables recibimos hoy en contacto directo a Luisa González quien conforma el binomio de la alianza revolución ciudadana reto junto a Diego Borja. La abogada de 47 años es la presidenta de revolución ciudadana, cargo que asumió un mes después de la derrota en segunda vuelta en las elecciones 2023. Dos años antes fue asambleista con UNES, fue funcionaria pública entre 2008 y 2017, encargos como Consul de Ecuador en Alicante, Ministra encargada de trabaj...


#### Enrique Gomez

Entrevista #1

In [ ]:
audio37 = procesar_audio_youtube("https://www.youtube.com/watch?v=j4LZPw3UFsc&list=PLwtmRZw8oM40RDj9c5VRIhDQN0mo0ANgb&index=12", "entrevista_Enrique_Gomez_Teleamazonas.mp4", "entrevista_limpia_Enrique_Gomez_Teleamazonas.wav")

Audio descargado como: entrevista_Enrique_Gomez_Teleamazonas.mp4
Audio procesado guardado en: entrevista_limpia_Enrique_Gomez_Teleamazonas.wav


In [ ]:
texto37 = transcribir_audio("entrevista_limpia_Enrique_Gomez_Teleamazonas.wav", "transcripcion_entrevista_Enrique_Gomez_Teleamazonas.txt")

Iniciando transcripción del archivo: entrevista_limpia_Enrique_Gomez_Teleamazonas.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Enrique_Gomez_Teleamazonas.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Saludos a Enrique Gomez, candidato a la presidencia por el Movimiento Suma. Gracias por acompañarnos. Usted es psicólogo clínico. Sí, efectivamente. Será lo que el país necesita un psicólogo. El país necesita soluciones y esa es nuestra propuesta. Desde la psicología he aprendido a escuchar a las personas, a escuchar sus problemas, pero sobre todo, buscar soluciones concretas que garanticen la mejor vida. ¿Cuál es la solución para este país? Bueno, para este país hoy es un desastre, definitivam...


Entrevista #2

In [ ]:
audio38 = procesar_audio_youtube("https://www.youtube.com/watch?v=vrvyYO_VcjA&list=PLhiHxCJzb9eYHj1Yu-wZKUShSuc6-jnBC&index=14", "entrevista_Enrique_Gomez_Ecuavisa.mp4", "entrevista_limpia_Enrique_Gomez_Ecuavisa.wav")

Audio descargado como: entrevista_Enrique_Gomez_Ecuavisa.mp4
Audio procesado guardado en: entrevista_limpia_Enrique_Gomez_Ecuavisa.wav


In [ ]:
texto38 = transcribir_audio("entrevista_limpia_Enrique_Gomez_Ecuavisa.wav", "transcripcion_entrevista_Enrique_Gomez_Ecuavisa.txt")

Iniciando transcripción del archivo: entrevista_limpia_Enrique_Gomez_Ecuavisa.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcripción guardada en: transcripcion_entrevista_Enrique_Gomez_Ecuavisa.txt

Primeras líneas de la transcripción:
--------------------------------------------------
 Hoy comenzamos el especial elecciones 2025, los presidenciables. Los 16 candidatos han sido invitados para conocer sus propuestas. Hoy es el turno del psicólogo clínico Enrique Gómez, quien a sus 38 años busca llegar a Carondelet en el binomio de suma que completa Inés Díaz. El candidato tiene una década en el sector público, comenzó en el correísmo, en la secretaría de gestión de riesgos y el Consejo de la Judicatura luego pasó al ECU 911 a la unidad de registro social que maneja los datos de ...


#### Daniel Noboa

Entrevista #1


In [ ]:
audio39 = procesar_audio_youtube("https://www.youtube.com/watch?v=S7qypKta608", "entrevista_Daniel_Noboa_DemocraciaTV.mp4", "entrevista_limpia_Daniel_Noboa_DemocraciaTV.wav")

Audio descargado como: entrevista_Daniel_Noboa_DemocraciaTV.mp4
Audio procesado guardado en: entrevista_limpia_Daniel_Noboa_DemocraciaTV.wav


In [ ]:
texto39 = transcribir_audio("entrevista_limpia_Daniel_Noboa_DemocraciaTV.wav", "transcripcion_entrevista_Daniel_Noboa_DemocraciaTV.txt")

Iniciando transcripción del archivo: entrevista_limpia_Daniel_Noboa_DemocraciaTV.wav


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


## Obtención de planes de Trabajo

Descargar y procesar información y los planes de trabajo de la página oficial del consejo nacional electoral. 

https://votoinformado.cne.gob.ec/

Intalación de Bibliotecas

In [1]:
# Se instala Selenium para la automatización de navegadores web.
#Request para la descarga de datos de sitios web y 
# Pandas para  la manipulación de los Dataframes.
!pip install selenium requests pandas


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
#Biblioteca usada para web scraping
!pip install beautifulsoup4


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Definición de Funciones

In [3]:
#Importación de librerías
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

Función que crea y configura WebDriver de Google Chrome usando Selenium para automatizar procesos.

In [ ]:

def create_driver():
    """
    Crea e inicializa un controlador (driver) de Chrome con configuraciones personalizadas 
    para evitar bloqueos y mejorar la automatización.

    Retorna:
    --------
    - webdriver.Chrome: Instancia del navegador Chrome con las opciones configuradas.
    """
    #Creación de objeto para configurar el comportamiento del navegador
    options = webdriver.ChromeOptions()
    #Evita la detección de Selenium por parte del sitio web para evitar bloqueo por bot
    options.add_argument('--disable-blink-features=AutomationControlled')
    #Evitar notificaciones emergentes
    options.add_argument('--disable-notifications')
    #Configuración de tamaño de pantalla
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--start-maximized')
    #Acepta automáticamente cualquier alerta
    options.set_capability('unhandledPromptBehavior', 'accept')
    
    #Crear la instancia de Chrome con las modificaciones
    driver = webdriver.Chrome(options=options)
    return driver

Función para manejar los dialogos emergentes de la página web haciendo click en Aceptar

In [ ]:
def handle_important_dialog(driver):
    """
    Maneja automáticamente un cuadro de diálogo importante en una página web, 
    haciendo clic en el botón "Aceptar" si está presente.

    Parámetros:
    -----------
    - driver (webdriver.Chrome): Instancia activa del navegador controlado por Selenium.

    Retorna:
    --------
    - bool: `True` si se hizo clic en el botón "Aceptar", `False` si no se encontró o hubo un error.
    """
    #Recibe el WebDriver abierto
    try:
        print("Esperando diálogo importante...")
        #Esperar 10 segpundos para que el botón "Aceptar" sea clickeable
        aceptar_button = WebDriverWait(driver, 10).until(
            #Ubicar el botón
            EC.element_to_be_clickable((By.XPATH, "//button[text()='Aceptar']"))
        )
        print("Haciendo click en Aceptar...")
        aceptar_button.click()
        time.sleep(2)
        return True
    except Exception as e:
        print(f"Error manejando diálogo importante: {str(e)}")
        return False

Función extrae la información de los binomios presidenciables desde la página web 

In [ ]:
def extract_binomio_info(driver):
    """
    Extrae información de los binomios presidenciales desde la página web del CNE.

    Parámetros:
    -----------
    - driver (webdriver.Chrome): Instancia activa del navegador controlado por Selenium.

    Retorna:
    --------
    - list[dict]: Lista de diccionarios con la información de los binomios encontrados. 
      Cada diccionario contiene:
        - 'Partido': Nombre del partido o alianza política.
        - 'Lista': Número de lista o alianza del binomio.
        - 'Presidente': Nombre del candidato a presidente.
        - 'Vicepresidente': Nombre del candidato a vicepresidente.
        - 'Plan_Trabajo': Estado de disponibilidad del plan de trabajo (Disponible/No disponible)."""
    
    info_list = []
    try:
        # Esperar por el contenedor principal
        print("Esperando contenedor principal...")
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "div.MuiContainer-root"))
        )
        time.sleep(2)
        
        # Encontrar todos los contenedores de binomios
        binomio_containers = driver.find_elements(By.CSS_SELECTOR, "div.MuiBox-root.css-1hj7a4p")
        print(f"\nEncontrados {len(binomio_containers)} binomios")

        #Itera cada binomio y crea un diccionario, se llena la información siguiente
        for idx, container in enumerate(binomio_containers, 1):
            try:
                print(f"\nProcesando binomio {idx}")
                
                info = {
                    'Partido': 'No encontrado',
                    'Lista': 'No encontrado',
                    'Presidente': 'No encontrado',
                    'Vicepresidente': 'No encontrado',
                    'Plan_Trabajo': 'No disponible'
                }
                
                # Obtener partido y lista desde el primer card
                partido_card = container.find_element(By.CSS_SELECTOR, "div.MuiCard-root")
                h5_elements = partido_card.find_elements(By.CSS_SELECTOR, "h5.MuiTypography-h5")
                
                #Encuentra la tarjeta del partido (MuiCard-root) y se buscan los elementos <h5> que se encuentra dentro
                if h5_elements:
                    info['Partido'] = h5_elements[0].text.strip()
                    if len(h5_elements) > 1:
                        texto = h5_elements[1].text.strip()
                        if "LISTA:" in texto:
                            info['Lista'] = texto.replace("LISTA:", "").strip()
                        elif "ALIANZA:" in texto:
                            # Extraer el primer número de la alianza (5-33 -> 5)
                            info['Lista'] = texto.split("-")[0].replace("ALIANZA:", "").strip()
                
                # Obtener presidente y vicepresidente del grid
                grid_container = container.find_element(By.CSS_SELECTOR, "div.MuiGrid-container")
                grid_items = grid_container.find_elements(By.CSS_SELECTOR, "div.MuiGrid-item")
                
                #Se encuentra la sección que contiene a los candidatos
                for item in grid_items:
                    try:
                        candidato_elements = item.find_elements(By.CSS_SELECTOR, "h6")
                        if len(candidato_elements) >= 2:
                            nombre = candidato_elements[0].text.strip()
                            cargo = candidato_elements[1].text.strip()
                            
                            if "PRESIDENTE" in cargo:
                                if "VICE" in cargo:
                                    info['Vicepresidente'] = nombre
                                else:
                                    info['Presidente'] = nombre
                    except:
                        continue
                
                # Verificar plan de trabajo
                plan_buttons = container.find_elements(By.CSS_SELECTOR, "button.MuiButton-root")
                if any("Plan de trabajo" in btn.text for btn in plan_buttons):
                    info['Plan_Trabajo'] = 'Disponible'
                
                # Solo agregar si tenemos información válida
                if (info['Partido'] != 'No encontrado' and info['Lista'] != 'No encontrado') or \
                   (info['Presidente'] != 'No encontrado' and info['Vicepresidente'] != 'No encontrado'):
                    info_list.append(info)
                    print("Información extraída:")
                    for key, value in info.items():
                        print(f"{key}: {value}")
                
            except Exception as e:
                print(f"Error procesando binomio {idx}: {str(e)}")
                continue
        
        print(f"\nTotal de binomios válidos encontrados: {len(info_list)}")
        
    except Exception as e:
        print(f"Error general extrayendo información: {str(e)}")
    
    return info_list

Función que automatiza la extracción de los datos de los candidatos desde el sitio web y se guarda el archivo en CSV.

In [ ]:
def scrape_cne_data():
    """
    Parámetros:
    -----------
    - No recibe parámetros.

    Retorna:
    --------
    - `pandas.DataFrame`: Contiene la información de los binomios presidenciales con las siguientes columnas:
        - `Partido`: Nombre del partido o alianza.
        - `Lista`: Número de lista.
        - `Presidente`: Nombre del candidato presidencial.
        - `Vicepresidente`: Nombre del candidato a la vicepresidencia.
        - `Plan_Trabajo`: Disponibilidad del plan de trabajo (Disponible/No disponible).

    """
    driver = create_driver()
    data = []
    
    try:
        print("Navegando a la página inicial...")
        driver.get("https://votoinformado.cne.gob.ec/")
        #Esperar 3 segundos a que se abra la página
        time.sleep(3)
        
        #cerrar mesajes emergentes
        if not handle_important_dialog(driver):
            return pd.DataFrame()
        
        # Click en proceso electoral
        print("Buscando proceso electoral...")
        proceso = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "h6.MuiTypography-root.MuiTypography-h6.css-1o1fs8j"))
        )
        proceso.click()
        time.sleep(3)
        
        # Click en presidente y vicepresidente usando JavaScript
        print("Buscando sección de presidente y vicepresidente...")
        presidente = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "h6.MuiTypography-root.MuiTypography-h6.css-zoebuu"))
        )
        driver.execute_script("arguments[0].click();", presidente)
        time.sleep(3)
        
        print("Extrayendo información de binomios...")
        data = extract_binomio_info(driver)
        
    except Exception as e:
        print(f"Error general: {str(e)}")
    
    finally:
        driver.quit()
    
    # Crear DataFrame
    df = pd.DataFrame(data)

    #Si el Dataframe no está vacío:
    #Lista se convierte a formato numérico
    
    if not df.empty:
        # Ordenar por número de lista
        df['Lista'] = pd.to_numeric(df['Lista'], errors='ignore')
        df = df.sort_values('Lista')
        
        # Guardar en CSV
        df.to_csv('binomios_presidenciales.csv', index=False, encoding='utf-8-sig')
        print("\nDatos guardados en binomios_presidenciales.csv")
    
    return df

Ejecución de la función scrape_cne_data y se imprimen los datos

In [8]:
if __name__ == "__main__":
    df = scrape_cne_data()
    print("\nResultados obtenidos:")
    print(df)

Navegando a la página inicial...
Esperando diálogo importante...
Haciendo click en Aceptar...
Buscando proceso electoral...
Buscando sección de presidente y vicepresidente...
Extrayendo información de binomios...
Esperando contenedor principal...

Encontrados 16 binomios

Procesando binomio 1
Información extraída:
Partido: MOVIMIENTO CENTRO DEMOCRÁTICO
Lista: 1
Presidente: JIMMY JAIRALA VALLAZZA
Vicepresidente: LUCIA VALLECILLA SUAREZ
Plan_Trabajo: Disponible

Procesando binomio 2
Información extraída:
Partido: PARTIDO UNIDAD POPULAR
Lista: 2
Presidente: JORGE ESCALA
Vicepresidente: PACHA TERAN
Plan_Trabajo: Disponible

Procesando binomio 3
Información extraída:
Partido: PARTIDO SOCIEDAD PATRIÓTICA 21 DE ENERO
Lista: 3
Presidente: ANDREA GONZALEZ
Vicepresidente: GALO MONCAYO
Plan_Trabajo: Disponible

Procesando binomio 4
Información extraída:
Partido: MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA "PID"
Lista: 4
Presidente: VICTOR ARAUS
Vicepresidente: CRISTINA CARRERA
Plan_Trabajo: Disponible


C:\Users\Sloth\AppData\Local\Temp\ipykernel_16684\221810200.py:48: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df['Lista'] = pd.to_numeric(df['Lista'], errors='ignore')


In [9]:
dataframe = df

DESCARGAR LOS PDFs DE LOS PLANES DE TRABAJO Y GUARDAR EN LA CARPETA PLANES DE TRABAJO

In [11]:
#Importación de librerías
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import os
import time
import glob
import shutil

Función que crea un WebDriver de Chrome que incluye la descarga automática de archivos en una carperta.

In [12]:
def create_driver(download_dir):
    options = webdriver.ChromeOptions()
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--disable-notifications')
    options.add_argument('--window-size=1920,1080')
    options.add_argument('--start-maximized')
    #Configuración de descarga automática
    prefs = {
        #Directorio de descarga
        "download.default_directory": download_dir,
        #No preguntar dónde guardar el archivo
        "download.prompt_for_download": False,
        #Permitir cambios en la carpeta de descargas
        "download.directory_upgrade": True,
        #Se habilita protección ante descargas peligrosas
        "safebrowsing.enabled": True
    }
    options.add_experimental_option("prefs", prefs)
    
    driver = webdriver.Chrome(options=options)
    return driver

Función para manejar el click en un elemento de diferentes maneras

In [13]:
def click_element_safely(driver, element, description="elemento"):
    """Intenta hacer clic en un elemento de múltiples maneras."""
    #Lista de métodos para hacer click
    methods = [
        lambda: element.click(),
        lambda: driver.execute_script("arguments[0].click();", element),
        lambda: ActionChains(driver).move_to_element(element).click().perform(),
        lambda: driver.execute_script("arguments[0].dispatchEvent(new MouseEvent('click', {bubbles: true, cancelable: true}));", element)
    ]

    #Bucle para intentar cada uno de los métodos
    for idx, method in enumerate(methods, 1):
        try:
            print(f"Intentando clic {idx} en {description}...")
            method()
            time.sleep(2)
            return True
        except Exception as e:
            print(f"Error en intento {idx}: {str(e)}")
            time.sleep(1)
    return False

Función que permite la espera de la descarga correcta de un archivo PDF 

In [14]:
def wait_for_download(directory, timeout=30):
    """Espera a que el archivo se descargue completamente."""
    #Guardar tiempo de inicio de la descarga
    start_time = time.time()
    #Verifica la descarga si el tiempo no ha llegado a su límite de 30s
    while time.time() - start_time < timeout:
        #detectar archivos en proceso de descarga
        downloading_files = glob.glob(os.path.join(directory, "*.crdownload")) + \
                          glob.glob(os.path.join(directory, "*.tmp")) + \
                          glob.glob(os.path.join(directory, "*.download"))
        #Busca en el directorio archivos .pdf para verificar descarga
        pdf_files = glob.glob(os.path.join(directory, "*.pdf"))
        
        #Verificar si la descarga ha terminado
        if not downloading_files and pdf_files:
            time.sleep(1)
            return pdf_files[0]
        time.sleep(0.5)
    return None

Función para automatizar la descarga de los planes de trabajo de los candidatos presidenciales 

In [15]:
def descargar_planes_trabajo():
    #Crear la carpeta "planes_trabajo" para guardar los archivos descargados
    base_dir = "planes_trabajo"
    #abs_path determina el path de descarga
    os.makedirs(base_dir, exist_ok=True)
    abs_path = os.path.abspath(base_dir)
    
    #Directorio temporal para el manejo de descargas temporales
    temp_dir = os.path.join(abs_path, "temp")
    os.makedirs(temp_dir, exist_ok=True)
    
    #Crear WebDriver
    driver = create_driver(temp_dir)
    exitos = []
    fallos = []
    
    #Manejo de diálogos emergentes
    try:
        print("Navegando a la página inicial...")
        driver.get("https://votoinformado.cne.gob.ec/")
        time.sleep(5)  # Cargar la página CNE y esperar 5s para asegurar la carga
        
        # Click en Aceptar inicial
        try:
            aceptar_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[text()='Aceptar']"))
            )
            click_element_safely(driver, aceptar_button, "botón Aceptar")
            time.sleep(3)
        except Exception as e:
            print(f"Error en diálogo inicial: {str(e)}")
            return
        
        #Navegar hasta la sección de presidentes y vicepresidentes
        # Click en proceso electoral con reintento
        print("Buscando proceso electoral...")
        max_attempts = 3
        for attempt in range(max_attempts):
            try:
                proceso = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h6.MuiTypography-root.MuiTypography-h6.css-1o1fs8j"))
                )
                if click_element_safely(driver, proceso, "proceso electoral"):
                    break
            except Exception as e:
                if attempt == max_attempts - 1:
                    print(f"No se pudo hacer clic en proceso electoral: {str(e)}")
                    return
                time.sleep(3)
        time.sleep(5)
        
        # Click en presidente y vicepresidente con reintento
        print("Buscando sección de presidente y vicepresidente...")
        #Encontrar binomios
        for attempt in range(max_attempts):
            try:
                presidente = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "h6.MuiTypography-root.MuiTypography-h6.css-zoebuu"))
                )
                if click_element_safely(driver, presidente, "presidente y vicepresidente"):
                    break
            except Exception as e:
                if attempt == max_attempts - 1:
                    print(f"No se pudo hacer clic en presidente y vicepresidente: {str(e)}")
                    return
                time.sleep(3)
        time.sleep(5)
        
        # Encontrar todos los binomios
        print("Esperando que carguen los binomios...")
        try:
            binomios = WebDriverWait(driver, 15).until(
                EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.MuiBox-root.css-1hj7a4p"))
            )
            print(f"\nEncontrados {len(binomios)} binomios")
        except Exception as e:
            print(f"Error encontrando binomios: {str(e)}")
            return
        
        #Limpia de carpeta temporal antes de la descarga
        # Procesar cada binomio
        for idx, binomio in enumerate(binomios, 1):
            try:
                # Limpiar directorio temporal
                for file in glob.glob(os.path.join(temp_dir, "*")):
                    try:
                        os.remove(file)
                    except:
                        pass
                
                # Obtener número de lista
                lista = "XX"
                try:
                    driver.execute_script("arguments[0].scrollIntoView({behavior: 'smooth', block: 'center'});", binomio)
                    time.sleep(1)
                    
                    h5_elements = binomio.find_elements(By.CSS_SELECTOR, "h5.MuiTypography-h5")
                    for h5 in h5_elements:
                        texto = h5.text.strip()  # Definimos texto aquí
                        print(f"Texto encontrado: {texto}") 
                        if "LISTA:" in texto:
                            lista = texto.replace("LISTA:", "").strip()
                            break
                        elif "ALIANZA:" in texto:
                            # Extraer el primer número de la alianza (5-33 -> 5)
                            lista = texto.split("-")[0].replace("ALIANZA:", "").strip()
                            break
                except:
                    pass
                
                print(f"\nProcesando binomio {idx} - Lista {lista}")
                
                # Buscar y hacer clic en el botón de plan de trabajo
                try:
                    plan_button = binomio.find_element(By.CSS_SELECTOR, "button.MuiButton-root")
                    if "Plan de trabajo" in plan_button.text:
                        if click_element_safely(driver, plan_button, f"plan de trabajo lista {lista}"):
                            # Esperar la descarga
                            downloaded_file = wait_for_download(temp_dir)
                            if downloaded_file:
                                final_name = os.path.join(abs_path, f"plan_trabajo_lista_{lista}.pdf")
                                shutil.move(downloaded_file, final_name)
                                print(f"✓ Plan de trabajo guardado exitosamente para Lista {lista}")
                                exitos.append(lista)
                            else:
                                print(f"× Timeout esperando descarga para Lista {lista}")
                                fallos.append(lista)
                        else:
                            print(f"× No se pudo hacer clic en el botón para Lista {lista}")
                            fallos.append(lista)
                except Exception as e:
                    print(f"Error con el botón de descarga para Lista {lista}: {str(e)}")
                    fallos.append(lista)
                
                time.sleep(3)
                
            except Exception as e:
                print(f"Error procesando binomio {idx}: {str(e)}")
                fallos.append(lista)
        
    except Exception as e:
        print(f"Error general: {str(e)}")

    #Eliminar la carpeta temporal
    finally:
        driver.quit()
        
        try:
            shutil.rmtree(temp_dir)
        except:
            pass
        
        #Imprimir un resumen de las descargas exitosas y fallidas
        print("\n=== RESUMEN DE DESCARGAS ===")
        print(f"Planes descargados exitosamente ({len(exitos)}): {', '.join(exitos)}")
        print(f"Planes con error ({len(fallos)}): {', '.join(fallos)}")
        
        #Verificación de archivos PDF guardados
        print("\n=== VERIFICACIÓN DE ARCHIVOS ===")
        archivos_encontrados = glob.glob(os.path.join(abs_path, "*.pdf"))
        print(f"Archivos encontrados en disco ({len(archivos_encontrados)}):")
        for archivo in archivos_encontrados:
            print(f"- {os.path.basename(archivo)} ({os.path.getsize(archivo)} bytes)")



Ejecución de la función para la descarga de los planes de trabajo

In [16]:
if __name__ == "__main__":
    descargar_planes_trabajo()

Navegando a la página inicial...
Intentando clic 1 en botón Aceptar...
Buscando proceso electoral...
Intentando clic 1 en proceso electoral...
Buscando sección de presidente y vicepresidente...
Intentando clic 1 en presidente y vicepresidente...
Esperando que carguen los binomios...

Encontrados 16 binomios
Texto encontrado: MOVIMIENTO CENTRO DEMOCRÁTICO
Texto encontrado: LISTA: 1

Procesando binomio 1 - Lista 1
Intentando clic 1 en plan de trabajo lista 1...
✓ Plan de trabajo guardado exitosamente para Lista 1
Texto encontrado: PARTIDO UNIDAD POPULAR
Texto encontrado: LISTA: 2

Procesando binomio 2 - Lista 2
Intentando clic 1 en plan de trabajo lista 2...
✓ Plan de trabajo guardado exitosamente para Lista 2
Texto encontrado: PARTIDO SOCIEDAD PATRIÓTICA 21 DE ENERO
Texto encontrado: LISTA: 3

Procesando binomio 3 - Lista 3
Intentando clic 1 en plan de trabajo lista 3...
✓ Plan de trabajo guardado exitosamente para Lista 3
Texto encontrado: MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA "PID"
Te

In [18]:
# Instalación de bibliotecas para el procesamiento de PDFs
# manipulación de datos en Excel
!pip install PyPDF2 pandas openpyxl


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
# Instalación de bibliotecas para el procesamiento
# extrección de texto y manipulación de archivos PDF e imagenes
!pip install pytesseract pdf2image PyPDF2 pandas openpyxl


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
#Importación de bibliotecas
import os
import glob
import pandas as pd
import PyPDF2
import pytesseract
from pdf2image import convert_from_path
import re
import codecs

#### LImpieza y normalización del texto extraido 

Función que permite la limpieza y normalización de texto: Eliminación de caracteres corruptos

In [22]:
def limpiar_texto(texto):
    """Limpia y normaliza el texto."""
    # Intentar diferentes codificaciones
    codificaciones = ['utf-8', 'latin1', 'cp1252']
    texto_limpio = texto
    
    for codec in codificaciones:
        #decodificar el texto que se encuentran en diferentes formatos
        try:
            if isinstance(texto, bytes):
                texto_limpio = texto.decode(codec)
            elif isinstance(texto, str):
                texto_limpio = texto.encode(codec).decode(codec)
            break
        except:
            continue
    
    # Eliminar caracteres no imprimibles y corruptos
    # Permite saltos de línea y tabulaciones
    texto_limpio = ''.join(char for char in texto_limpio if char.isprintable() or char in '\n\t')
    
    return texto_limpio

Función para la extracción de texto de una imagen utilizando OCR con Tesseract

In [23]:
def extraer_texto_imagen(imagen):
    """Extrae texto de una imagen usando OCR."""
    try:
        # Configurar Tesseract
        custom_config = r'--oem 3 --psm 6 -l spa'
        #Extraer el texto de la imagen usando parámetros configurados
        texto = pytesseract.image_to_string(imagen, config=custom_config)
        return texto
    except Exception as e:
        print(f"Error en OCR: {str(e)}")
        return ""


Función para la lectura de archivos PDF, extracción de texto y el uso de OCR

In [24]:
def leer_pdf(ruta_archivo):
    """Lee un archivo PDF intentando primero como texto y luego como imagen si es necesario."""
    texto_completo = ""
    
    #abrir archivo PDF para modo de lectura binaria
    try:
        # Primer intento: leer como texto normal
        with open(ruta_archivo, 'rb') as file:
            #Recorre cada página y extrae el texto
            pdf_reader = PyPDF2.PdfReader(file)
            texto = ""
            for page in pdf_reader.pages:
                try:
                    texto += page.extract_text() + "\n"
                except:
                    continue
            
            # Si el texto está vacío o tiene muy pocos caracteres, intentar OCR
            if len(texto.strip()) < 100:
                print("Texto insuficiente, intentando OCR...")
                # Convertir PDF a imágenes
                imagenes = convert_from_path(ruta_archivo, 300)  # Mayor DPI para mejor calidad
                #Procesa cada página con OCR
                for pagina, imagen in enumerate(imagenes, 1):
                    print(f"Procesando página {pagina} con OCR...")
                    texto_ocr = extraer_texto_imagen(imagen)
                    #Concatena el texto extraído en texto_completo
                    texto_completo += texto_ocr + "\n"
                    
                print(f"OCR completado: {len(texto_completo)} caracteres extraídos")
            else:
                texto_completo = texto
            
        return limpiar_texto(texto_completo)
    
    # En caso de error imprime mensaje y devuelve una cadena vacía    
    except Exception as e:
        print(f"Error leyendo PDF: {str(e)}")
        return ""

Función para la extracción de número de listas desde el nombre del archivo de "plan_trabajo_lista_#"

In [25]:
def extraer_numero_lista(archivo):
    """Extrae el número de lista desde el nombre del archivo."""
    try:
        # Obtener solo el nombre del archivo sin la ruta
        nombre_archivo = os.path.basename(archivo)

        # Usar una expresión regular para extraer el número de lista
        match = re.search(r"plan_trabajo_lista_(\d+)", nombre_archivo)

        # Si se encuentra un número, retornarlo como entero
        if match:
            return int(match.group(1))
        else:
            return None  # Retorna None si no se encuentra un número válido
    except Exception as e:
        print(f"Error extrayendo número de lista: {str(e)}")
        return None

Función para la extracción de contenido y la generación de un resumen en Excel de los archivos PDF de los planes de trabajo

In [26]:
def procesar_planes_trabajo():
    # Directorio donde están los PDFs
    directorio = "planes_trabajo"
    
    # Lista para almacenar los datos
    datos = []
    contenidos = {}
    
    # Buscar todos los archivos PDF
    archivos_pdf = glob.glob(os.path.join(directorio, "*.pdf"))
    total_archivos = len(archivos_pdf)
    print(f"Encontrados {total_archivos} archivos PDF")
    
    #Recorre cada archivo PDF mostrando el progreso 
    for idx, archivo in enumerate(archivos_pdf, 1):
        print(f"\nProcesando archivo {idx}/{total_archivos}: {os.path.basename(archivo)}")
        
        #Extraer el numero de lista desde el nombre del archivo
        try:
            # Obtener número de lista
            lista = extraer_numero_lista(archivo)
            
            # Leer contenido del PDF
            contenido = leer_pdf(archivo)
            
            # Obtener tamaño del archivo
            tamaño = os.path.getsize(archivo)
            
            # Guardar datos principales
            datos.append({
                'Lista': lista,
                'Nombre_Archivo': os.path.basename(archivo),
                'Tamaño_Bytes': tamaño,
                'Num_Caracteres': len(contenido),
                'Contenido_Extraido': len(contenido.strip()) > 0
            })
            
            # Guardar contenido en diccionario separado
            contenidos[lista] = contenido
            
            print(f"✓ Procesado exitosamente - {tamaño:,} bytes, {len(contenido):,} caracteres")
            
        except Exception as e:
            print(f"× Error procesando archivo: {str(e)}")
    
    # Crear DataFrame
    df = pd.DataFrame(datos)
    
    # Ordenar por número de lista
    df['Lista'] = pd.to_numeric(df['Lista'], errors='coerce')
    df = df.sort_values('Lista')
    
    # Guardar DataFrame en Excel
    nombre_excel = "planes_trabajo_resumen.xlsx"
    df.to_excel(nombre_excel, index=False)
    print(f"\nResumen guardado en {nombre_excel}")
    
    # Guardar contenidos en archivos de texto separados
    contenidos_dir = "planes_trabajo_texto"
    os.makedirs(contenidos_dir, exist_ok=True)
    
    for lista, contenido in contenidos.items():
        nombre_archivo = os.path.join(contenidos_dir, f"plan_trabajo_lista_{lista}.txt")
        with open(nombre_archivo, 'w', encoding='utf-8') as f:
            f.write(contenido)
    
    print(f"Contenidos guardados en carpeta {contenidos_dir}/")
    
    # Mostrar resumen
    print("\n=== RESUMEN DE PROCESAMIENTO ===")
    print(f"Total archivos procesados: {len(df)}")
    print("\nDetalle de archivos:")
    print(df[['Lista', 'Tamaño_Bytes', 'Num_Caracteres', 'Contenido_Extraido']].to_string())
    
    return df, contenidos

Ejecución de la función "procesar_planes_trabajo" para el procesamiento de los documentos PDF de las propuestas presindenciales

In [27]:
if __name__ == "__main__":
    # Instalar dependencias necesarias
    try:
        import pytesseract
        import pdf2image
    except ImportError:
        print("Instalando dependencias requeridas...")
        import subprocess
        subprocess.run(["pip", "install", "pytesseract", "pdf2image", "PyPDF2", "pandas", "openpyxl"])
    
    # Verificar que Tesseract esté instalado
    try:
        pytesseract.get_tesseract_version()
    except Exception:
        print("Error: Tesseract no está instalado.")
        print("Por favor, instala Tesseract OCR:")
        print("- Windows: https://github.com/UB-Mannheim/tesseract/wiki")
        print("- Linux: sudo apt-get install tesseract-ocr")
        print("- Mac: brew install tesseract")
        exit(1)
    
    df, contenidos = procesar_planes_trabajo()

Encontrados 16 archivos PDF

Procesando archivo 1/16: plan_trabajo_lista_1.pdf
✓ Procesado exitosamente - 1,331,710 bytes, 172,894 caracteres

Procesando archivo 2/16: plan_trabajo_lista_12.pdf
✓ Procesado exitosamente - 1,133,348 bytes, 245,186 caracteres

Procesando archivo 3/16: plan_trabajo_lista_16.pdf
✓ Procesado exitosamente - 1,642,100 bytes, 154,622 caracteres

Procesando archivo 4/16: plan_trabajo_lista_17.pdf
✓ Procesado exitosamente - 930,659 bytes, 82,597 caracteres

Procesando archivo 5/16: plan_trabajo_lista_18.pdf
✓ Procesado exitosamente - 1,040,747 bytes, 84,322 caracteres

Procesando archivo 6/16: plan_trabajo_lista_2.pdf
✓ Procesado exitosamente - 2,179,945 bytes, 64,566 caracteres

Procesando archivo 7/16: plan_trabajo_lista_20.pdf
Texto insuficiente, intentando OCR...
Procesando página 1 con OCR...
Error en OCR: (1, 'Error opening data file C:\\Program Files\\Tesseract-OCR/tessdata/spa.traineddata Please make sure the TESSDATA_PREFIX environment variable is set to

### Creación de Dataframes

DATAFRAME CON PLANES DE TRABAJO Y DATAFRAME CON BIOGRAFIAS:

In [41]:
import os
import pandas as pd

# Leer el CSV original dos veces (uno para cada dataframe)
df_biografias = pd.read_csv('binomios_presidenciales.csv')
df_planes = pd.read_csv('binomios_presidenciales.csv')

In [42]:
# Directorios donde están los archivos
directorio_planes = "planes_trabajo_texto/"
directorio_biografias = "biografias/"

In [43]:
# Crear diccionarios para almacenar los textos
planes_trabajo = {}
biografias = {}

In [44]:
# Leer los archivos de planes de trabajo
for archivo in os.listdir(directorio_planes):
    if archivo.startswith("plan_de_trabajo_lista_") and archivo.endswith(".txt"):
        try:
            num_lista = int(archivo.replace("plan_de_trabajo_lista_", "").replace(".txt", "").strip())
            with open(os.path.join(directorio_planes, archivo), "r", encoding="utf-8") as f:
                contenido = f.read()
                planes_trabajo[num_lista] = contenido
        except Exception as e:
            print(f"Error con archivo de plan de trabajo {archivo}: {e}")


In [45]:
# Leer los archivos de biografías
for archivo in os.listdir(directorio_biografias):
    if archivo.startswith("biografia_lista_") and archivo.endswith(".txt"):
        try:
            num_lista = int(archivo.replace("biografia_lista_", "").replace(".txt", "").strip())
            with open(os.path.join(directorio_biografias, archivo), "r", encoding="utf-8") as f:
                contenido = f.read()
                biografias[num_lista] = contenido
        except Exception as e:
            print(f"Error con archivo de biografía {archivo}: {e}")

In [46]:

# Convertir la columna Lista a entero en ambos dataframes
df_biografias["Lista"] = df_biografias["Lista"].astype(int)
df_planes["Lista"] = df_planes["Lista"].astype(int)

# Para el dataframe de biografías
if "Plan_Trabajo" in df_biografias.columns:
    df_biografias = df_biografias.drop("Plan_Trabajo", axis=1)  # Eliminar columna Plan_Trabajo
df_biografias["Biografia"] = df_biografias["Lista"].map(biografias)  # Añadir columna Biografia


In [47]:
# Para el dataframe de planes
df_planes["Plan_Trabajo"] = df_planes["Lista"].map(planes_trabajo)

# Guardar los dos CSV
print("Guardando archivos CSV...")

# Guardar dataframe con biografías
df_biografias.to_csv("dataframe_con_biografias.csv", index=False, encoding='utf-8')
print("Archivo guardado como dataframe_con_biografias.csv")

# Guardar dataframe con planes de trabajo
df_planes.to_csv("dataframe_con_planes.csv", index=False, encoding='utf-8')
print("Archivo guardado como dataframe_con_planes.csv")


Guardando archivos CSV...
Archivo guardado como dataframe_con_biografias.csv
Archivo guardado como dataframe_con_planes.csv


In [48]:
# Imprimir resumen
print("\nResumen:")
print(f"Total de planes de trabajo encontrados: {len(planes_trabajo)}")
print(f"Total de biografías encontradas: {len(biografias)}")
print("Listas encontradas:")
print(f"- Planes de trabajo: {sorted(planes_trabajo.keys())}")
print(f"- Biografías: {sorted(biografias.keys())}")


Resumen:
Total de planes de trabajo encontrados: 16
Total de biografías encontradas: 16
Listas encontradas:
- Planes de trabajo: [1, 2, 3, 4, 5, 6, 7, 8, 12, 16, 17, 18, 20, 21, 23, 25]
- Biografías: [1, 2, 3, 4, 5, 6, 7, 8, 12, 16, 17, 18, 20, 21, 23, 25]


In [33]:
df_planes

,Partido,Lista,Presidente,Vicepresidente,Plan_Trabajo
0,MOVIMIENTO CENTRO DEMOCRÁTICO,1,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,PAGINA 1\n 1. Introducción\nEcuador es un país...
1,PARTIDO UNIDAD POPULAR,2,JORGE ESCALA,PACHA TERAN,PAGINA 3\n\nIntroducción:\nEl Ecuador vive una...
2,PARTIDO SOCIEDAD PATRIÓTICA 21 DE ENERO,3,ANDREA GONZALEZ,GALO MONCAYO,PAGINA 1\n\n1. DIAGNÓSTICO DE LA SITUACIÓN ACT...
3,"MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA ""PID""",4,VICTOR ARAUS,CRISTINA CARRERA,PAGINA 4\n1. Introducción\nEl presente Plan de...
4,REVOLUCIÓN CIUDADANA - RETO,5,LUISA GONZALEZ,DIEGO BORJA,PAGINA 7\n\nPrograma de Gobierno de la Revoluc...
5,PARTIDO SOCIAL CRISTIANO,6,HENRY KRONFLE KOZHAYA,DALLYANA PASSAILAIGUE,PAGINA 3\n PLAN DE TRABAJO\nPARTIDO SOCIAL CRI...
6,"MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN",7,DANIEL NOBOA AZIN,MARIA JOSE PINTO,PAGINA 5 1. INTRODUCCIÓN\nCoincidir con un gru...
7,PARTIDO AVANZA,8,LUIS FELIPE TILLERIA,KARLA PAULINA ROSERO,PAGINA 2\nPROPUESTA LA GRAN DEVOLUCIÓN\n1. DIA...
8,PARTIDO IZQUIERDA DEMOCRÁTICA,12,CARLOS RABASCALL,ALEJANDRA RIVAS MANTILLA,PAGINA 6 \n 1. INTRODUCCIÓN\nEcuador se encuen...
9,"MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPEND...",16,JUAN IVAN CUEVA,CRISTINA REYES,PAGINA 1\n 1.- INTRODUCCION\nEl Movimiento AMI...


In [34]:
df_biografias

,Partido,Lista,Presidente,Vicepresidente,Biografia
0,MOVIMIENTO CENTRO DEMOCRÁTICO,1,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,Jimmy Jairala y Lucía Vallecilla – Movimiento ...
1,PARTIDO UNIDAD POPULAR,2,JORGE ESCALA,PACHA TERAN,Jorge Escala y Lucía Terán – Partido Unidad Po...
2,PARTIDO SOCIEDAD PATRIÓTICA 21 DE ENERO,3,ANDREA GONZALEZ,GALO MONCAYO,Andrea González y Galo Moncayo – Partido Socie...
3,"MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA ""PID""",4,VICTOR ARAUS,CRISTINA CARRERA,Víctor Araus y Cristina Carrera – Movimiento P...
4,REVOLUCIÓN CIUDADANA - RETO,5,LUISA GONZALEZ,DIEGO BORJA,Luisa González – Revolución Ciudadana (RC) y R...
5,PARTIDO SOCIAL CRISTIANO,6,HENRY KRONFLE KOZHAYA,DALLYANA PASSAILAIGUE,Henry Kronfle y Dallyana Passailaigue – Partid...
6,"MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN",7,DANIEL NOBOA AZIN,MARIA JOSE PINTO,Daniel Noboa – Movimiento Acción Democrática N...
7,PARTIDO AVANZA,8,LUIS FELIPE TILLERIA,KARLA PAULINA ROSERO,Luis Felipe Tillería y Karla Rosero – Partido ...
8,PARTIDO IZQUIERDA DEMOCRÁTICA,12,CARLOS RABASCALL,ALEJANDRA RIVAS MANTILLA,Carlos Rabascall y María Rivas – Partido Izqui...
9,"MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPEND...",16,JUAN IVAN CUEVA,CRISTINA REYES,Juan Cueva y Cristina Reyes – Movimiento Amigo...


DATAFRAME CON ENTREVISTAS:

In [49]:
import os
import pandas as pd

# Leer el CSV original
df_entrevistas = pd.read_csv('binomios_presidenciales.csv')

# Directorios donde están los archivos
directorio_entrevistas = "entrevistas/"
directorio_descripciones = "descripciones/"
directorio_temas = "temas/"


In [50]:
# Crear diccionarios para almacenar los textos
entrevistas = {}
descripciones = {}
temas = {}

In [52]:
# Leer los archivos de entrevistas
print("Leyendo entrevistas...")
for archivo in os.listdir(directorio_entrevistas):
    if archivo.startswith("entrevista_lista_") and archivo.endswith(".txt"):
        try:
            num_lista = int(archivo.replace("entrevista_lista_", "").replace(".txt", "").strip())
            with open(os.path.join(directorio_entrevistas, archivo), "r", encoding="utf-8") as f:
                contenido = f.read()
                entrevistas[num_lista] = contenido
        except Exception as e:
            print(f"Error con archivo de entrevista {archivo}: {e}")

# Leer los archivos de descripciones
print("Leyendo descripciones...")
for archivo in os.listdir(directorio_descripciones):
    if archivo.startswith("descripcion_entrevista_lista_") and archivo.endswith(".txt"):
        try:
            num_lista = int(archivo.replace("descripcion_entrevista_lista_", "").replace(".txt", "").strip())
            with open(os.path.join(directorio_descripciones, archivo), "r", encoding="utf-8") as f:
                contenido = f.read()
                descripciones[num_lista] = contenido
        except Exception as e:
            print(f"Error con archivo de descripción {archivo}: {e}")

# Leer los archivos de temas
print("Leyendo temas...")
for archivo in os.listdir(directorio_temas):
    if archivo.startswith("temas_entrevista_lista_") and archivo.endswith(".txt"):
        try:
            num_lista = int(archivo.replace("temas_entrevista_lista_", "").replace(".txt", "").strip())
            with open(os.path.join(directorio_temas, archivo), "r", encoding="utf-8") as f:
                contenido = f.read()
                temas[num_lista] = contenido
        except Exception as e:
            print(f"Error con archivo de temas {archivo}: {e}")


Leyendo entrevistas...
Leyendo descripciones...
Leyendo temas...


In [53]:
# Convertir la columna Lista a entero
df_entrevistas["Lista"] = df_entrevistas["Lista"].astype(int)

# Eliminar columnas que no necesitamos (si existen)
columnas_a_mantener = ['Partido', 'Lista', 'Presidente', 'Vicepresidente']
df_entrevistas = df_entrevistas[columnas_a_mantener].copy()

In [54]:
# Añadir las nuevas columnas
df_entrevistas["Entrevistas"] = df_entrevistas["Lista"].map(entrevistas)
df_entrevistas["Descripciones"] = df_entrevistas["Lista"].map(descripciones)
df_entrevistas["Temas"] = df_entrevistas["Lista"].map(temas)

# Guardar el CSV
print("Guardando archivo CSV...")
df_entrevistas.to_csv("dataframe_con_entrevistas.csv", index=False, encoding='utf-8')
print("Archivo guardado como dataframe_con_entrevistas.csv")

# Imprimir resumen
print("\nResumen:")
print(f"Total de entrevistas encontradas: {len(entrevistas)}")
print(f"Total de descripciones encontradas: {len(descripciones)}")
print(f"Total de temas encontrados: {len(temas)}")
print("\nListas encontradas:")
print(f"- Entrevistas: {sorted(entrevistas.keys())}")
print(f"- Descripciones: {sorted(descripciones.keys())}")
print(f"- Temas: {sorted(temas.keys())}")

Guardando archivo CSV...
Archivo guardado como dataframe_con_entrevistas.csv

Resumen:
Total de entrevistas encontradas: 16
Total de descripciones encontradas: 16
Total de temas encontrados: 16

Listas encontradas:
- Entrevistas: [1, 2, 3, 4, 5, 6, 7, 8, 12, 16, 17, 18, 20, 21, 23, 25]
- Descripciones: [1, 2, 3, 4, 5, 6, 7, 8, 12, 16, 17, 18, 20, 21, 23, 25]
- Temas: [1, 2, 3, 4, 5, 6, 7, 8, 12, 16, 17, 18, 20, 21, 23, 25]


In [39]:
df_entrevistas

,Partido,Lista,Presidente,Vicepresidente,Entrevistas,Descripciones,Temas
0,MOVIMIENTO CENTRO DEMOCRÁTICO,1,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,ENTREVISTA 1\n\nEntrevista a Jimmy Jairala – E...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,"TEMAS ENTREVISTA 1\n\nTrayectoria política, Ce..."
1,PARTIDO UNIDAD POPULAR,2,JORGE ESCALA,PACHA TERAN,ENTREVISTA 1\n\nEntrevista a Jorge Escala – Ec...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,"TEMAS ENTREVISTA 1\n\nSalario mínimo, costo de..."
2,PARTIDO SOCIEDAD PATRIÓTICA 21 DE ENERO,3,ANDREA GONZALEZ,GALO MONCAYO,ENTREVISTA 1\n\nEntrevista a Andrea González N...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,TEMAS ENTREVISTA 1\n\nUso del petróleo para de...
3,"MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA ""PID""",4,VICTOR ARAUS,CRISTINA CARRERA,ENTREVISTA 1\n\nEntrevista a Victor Arauz - Te...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,TEMAS ENTREVISTA 1\n\nConspiración en su contr...
4,REVOLUCIÓN CIUDADANA - RETO,5,LUISA GONZALEZ,DIEGO BORJA,ENTREVISTA 1\n\nEntrevista a Luisa González - ...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,TEMAS ENTREVISTA 1\n\nSeguridad y plan Protege...
5,PARTIDO SOCIAL CRISTIANO,6,HENRY KRONFLE KOZHAYA,DALLYANA PASSAILAIGUE,ENTREVISTA 1\n\nEntrevista a Henry Kronfle – E...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,"TEMAS ENTREVISTA 1\n\nReducción del IVA, contr..."
6,"MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN",7,DANIEL NOBOA AZIN,MARIA JOSE PINTO,ENTREVISTA 1\n\nEntrevista a Daniel Noboa\n\nL...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,TEMAS ENTREVISTA 1\n\nSeguridad y lucha contra...
7,PARTIDO AVANZA,8,LUIS FELIPE TILLERIA,KARLA PAULINA ROSERO,ENTREVISTA 1\n\nEntrevista a Luis Felipe Tille...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,TEMAS ENTREVISTA 1\n\nHistorial y cuestionamie...
8,PARTIDO IZQUIERDA DEMOCRÁTICA,12,CARLOS RABASCALL,ALEJANDRA RIVAS MANTILLA,ENTREVISTA 1\n\nEntrevista a Carlos Rabascall ...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,TEMAS ENTREVISTA 1\n\nResiliencia y actitud en...
9,"MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPEND...",16,JUAN IVAN CUEVA,CRISTINA REYES,ENTREVISTA 1\n\nEntrevista a Juan Iván Cueva –...,DESCRIPCION ENTREVISTA 1\n\nEn esta entrevista...,TEMAS ENTREVISTA 1\n\nMotivación para ser cand...


In [57]:
df_biografias

,Partido,Lista,Presidente,Vicepresidente,Biografia
0,MOVIMIENTO CENTRO DEMOCRÁTICO,1,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,Jimmy Jairala y Lucía Vallecilla – Movimiento ...
1,PARTIDO UNIDAD POPULAR,2,JORGE ESCALA,PACHA TERAN,Jorge Escala y Lucía Terán – Partido Unidad Po...
2,PARTIDO SOCIEDAD PATRIÓTICA 21 DE ENERO,3,ANDREA GONZALEZ,GALO MONCAYO,Andrea González y Galo Moncayo – Partido Socie...
3,"MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA ""PID""",4,VICTOR ARAUS,CRISTINA CARRERA,Víctor Araus y Cristina Carrera – Movimiento P...
4,REVOLUCIÓN CIUDADANA - RETO,5,LUISA GONZALEZ,DIEGO BORJA,Luisa González – Revolución Ciudadana (RC) y R...
5,PARTIDO SOCIAL CRISTIANO,6,HENRY KRONFLE KOZHAYA,DALLYANA PASSAILAIGUE,Henry Kronfle y Dallyana Passailaigue – Partid...
6,"MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN",7,DANIEL NOBOA AZIN,MARIA JOSE PINTO,Daniel Noboa – Movimiento Acción Democrática N...
7,PARTIDO AVANZA,8,LUIS FELIPE TILLERIA,KARLA PAULINA ROSERO,Luis Felipe Tillería y Karla Rosero – Partido ...
8,PARTIDO IZQUIERDA DEMOCRÁTICA,12,CARLOS RABASCALL,ALEJANDRA RIVAS MANTILLA,Carlos Rabascall y María Rivas – Partido Izqui...
9,"MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPEND...",16,JUAN IVAN CUEVA,CRISTINA REYES,Juan Cueva y Cristina Reyes – Movimiento Amigo...


In [58]:
df_planes

,Partido,Lista,Presidente,Vicepresidente,Plan_Trabajo
0,MOVIMIENTO CENTRO DEMOCRÁTICO,1,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,PAGINA 1\n 1. Introducción\nEcuador es un país...
1,PARTIDO UNIDAD POPULAR,2,JORGE ESCALA,PACHA TERAN,PAGINA 3\n\nIntroducción:\nEl Ecuador vive una...
2,PARTIDO SOCIEDAD PATRIÓTICA 21 DE ENERO,3,ANDREA GONZALEZ,GALO MONCAYO,PAGINA 1\n\n1. DIAGNÓSTICO DE LA SITUACIÓN ACT...
3,"MOVIMIENTO PUEBLO IGUALDAD DEMOCRACIA ""PID""",4,VICTOR ARAUS,CRISTINA CARRERA,PAGINA 4\n1. Introducción\nEl presente Plan de...
4,REVOLUCIÓN CIUDADANA - RETO,5,LUISA GONZALEZ,DIEGO BORJA,PAGINA 7\n\nPrograma de Gobierno de la Revoluc...
5,PARTIDO SOCIAL CRISTIANO,6,HENRY KRONFLE KOZHAYA,DALLYANA PASSAILAIGUE,PAGINA 3\n PLAN DE TRABAJO\nPARTIDO SOCIAL CRI...
6,"MOVIMIENTO ACCION DEMOCRATICA NACIONAL, ADN",7,DANIEL NOBOA AZIN,MARIA JOSE PINTO,PAGINA 5 1. INTRODUCCIÓN\nCoincidir con un gru...
7,PARTIDO AVANZA,8,LUIS FELIPE TILLERIA,KARLA PAULINA ROSERO,PAGINA 2\nPROPUESTA LA GRAN DEVOLUCIÓN\n1. DIA...
8,PARTIDO IZQUIERDA DEMOCRÁTICA,12,CARLOS RABASCALL,ALEJANDRA RIVAS MANTILLA,PAGINA 6 \n 1. INTRODUCCIÓN\nEcuador se encuen...
9,"MOVIMIENTO AMIGO, ACCIÓN MOVILIZADORA INDEPEND...",16,JUAN IVAN CUEVA,CRISTINA REYES,PAGINA 1\n 1.- INTRODUCCION\nEl Movimiento AMI...


# 1. Preprocesamiento 

### PREPROCESAMIENTO DE PLANES DE TRABAJO:

In [101]:
# Importar librerías
import pandas as pd
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import unicodedata

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sloth\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sloth\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Función para eliminación de tíldes

In [ ]:
#Función para eliminar tildes
def eliminar_tildes(texto):
    """Elimina tildes y caracteres especiales"""
    return ''.join(c for c in unicodedata.normalize('NFD', texto)
                  if unicodedata.category(c) != 'Mn')

Función de limpieza

In [ ]:

def limpiar_texto(texto):
    """Función para limpiar y normalizar texto"""
    if not isinstance(texto, str):
        return ""
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar tildes
    texto = eliminar_tildes(texto)
    # Eliminar caracteres especiales y números
    texto = re.sub(r'[^\w\s]', ' ', texto)
    texto = re.sub(r'\d+', ' ', texto)
    # Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

Función verifica un título numérico

In [61]:
def es_titulo_numerico(oracion):
    """Verifica si la oración es un título con número al inicio"""
    patron_titulo = r'^\d+\.?\s+[A-ZÁÉÍÓÚÑ]'
    return bool(re.match(patron_titulo, oracion))

Función para el preprocesamiento del texto del plan de trabajo y la extracción de oraciones válidas

In [62]:
def procesar_plan_trabajo(row):
    """Procesa el plan de trabajo de una fila y retorna un DataFrame con las oraciones"""
    resultados = []
    stopwords_spanish = set(stopwords.words('spanish'))
    
    texto = row['Plan_Trabajo']
    if not isinstance(texto, str):
        return pd.DataFrame()
    
    # Dividir por páginas
    paginas = re.split(r'PAGINA \d+', texto)
    
    # Eliminar la primera entrada vacía si existe
    if paginas and not paginas[0].strip():
        paginas = paginas[1:]
    
    id_contador = 1  # Contador global para las oraciones
    
    for num_pagina, contenido_pagina in enumerate(paginas, 1):
        # Obtener oraciones de la página
        oraciones = sent_tokenize(contenido_pagina.strip())
        
        for oracion in oraciones:
            oracion = oracion.strip()
            # Verificar si la oración es válida y no es un título numérico
            if oracion and (oracion[0].isupper() or oracion.isupper()) and not es_titulo_numerico(oracion):
                # Limpiar la oración
                oracion_limpia = limpiar_texto(oracion)
                
                # Generar versión sin stopwords
                palabras = word_tokenize(oracion_limpia)
                sin_stopwords = ' '.join([palabra for palabra in palabras if palabra not in stopwords_spanish])
                
                # Solo agregar si la oración limpia no está vacía
                if oracion_limpia:
                    resultados.append({
                        'Lista': row['Lista'],
                        'Partido': row['Partido'],
                        'Presidente': row['Presidente'],
                        'Vicepresidente': row['Vicepresidente'],
                        'id_oracion': f"{row['Lista']}_{num_pagina}_{id_contador}",
                        'pagina': num_pagina,
                        'oracion_original': oracion,
                        'oracion_limpia': oracion_limpia,
                        'oracion_sin_stopwords': sin_stopwords
                    })
                    id_contador += 1
    
    return pd.DataFrame(resultados)

Lectura del datagrame con los planes de trabajo

In [63]:
# Leer el CSV con los planes de trabajo
print("Leyendo dataframe_con_planes.csv...")
df_planes = pd.read_csv("dataframe_con_planes.csv")

Leyendo dataframe_con_planes.csv...


Procesamiento de todos los planes de trabajo que están en df_planes usando la funció procesar_plan_trabajo a cada fila y guarda los resultados en una lista de DataFrames

In [64]:
# Lista para almacenar todos los DataFrames procesados
dfs_procesados = []

# Procesar cada plan de trabajo
print("Procesando planes de trabajo...")
for _, row in df_planes.iterrows():
    try:
        df_procesado = procesar_plan_trabajo(row)
        if not df_procesado.empty:
            dfs_procesados.append(df_procesado)
    except Exception as e:
        print(f"Error procesando plan de trabajo de Lista {row['Lista']}: {e}")

Procesando planes de trabajo...


Combinar todos los Dataframes en una sola tabla 

In [65]:
# Combinar todos los DataFrames
print("Combinando resultados...")
df_final = pd.concat(dfs_procesados, ignore_index=True)


Combinando resultados...


Guardar el Dataframe procesado en un archivo CSV

In [66]:
# Guardar el DataFrame procesado
print("Guardando resultados...")
df_final.to_csv("planes_trabajo_procesados.csv", index=False, encoding='utf-8')

Guardando resultados...


In [67]:
# Mostrar una muestra del DataFrame
print("\nMuestra del DataFrame procesado:")
print(df_final.head(3))

print("\nEstadísticas del procesamiento:")
print(f"Total de oraciones procesadas: {len(df_final)}")
print(f"Número de planes de trabajo procesados: {len(df_final['Lista'].unique())}")
print(f"Promedio de oraciones por plan: {len(df_final) / len(df_final['Lista'].unique()):.2f}")


Muestra del DataFrame procesado:
   Lista                        Partido              Presidente  \
0      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   
1      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   
2      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   

            Vicepresidente id_oracion  pagina  \
0  LUCIA VALLECILLA SUAREZ      1_1_1       1   
1  LUCIA VALLECILLA SUAREZ      1_1_2       1   
2  LUCIA VALLECILLA SUAREZ      1_1_3       1   

                                    oracion_original  \
0  Introducción\nEcuador es un país con una profu...   
1  Existen 14\nnacionalidades, 18 pueblos y 3 pue...   
2  Por sus particulares características geográfic...   

                                      oracion_limpia  \
0  introduccion ecuador es un pais con una profun...   
1  existen nacionalidades pueblos y pueblos indig...   
2  por sus particulares caracteristicas geografic...   

                               oracion_sin_stopwor

In [68]:
# Mostrar distribución de oraciones por página
print("\nDistribución de oraciones por página:")
print(df_final.groupby(['Lista', 'pagina'])['id_oracion'].count())


Distribución de oraciones por página:
Lista  pagina
1      1         19
       2          9
       3         17
       4         15
       5          8
                 ..
25     121        8
       122        6
       123        6
       124        8
       125        4
Name: id_oracion, Length: 1325, dtype: int64


In [69]:
df_final

,Lista,Partido,Presidente,Vicepresidente,id_oracion,pagina,oracion_original,oracion_limpia,oracion_sin_stopwords
0,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_1,1,Introducción\nEcuador es un país con una profu...,introduccion ecuador es un pais con una profun...,introduccion ecuador pais profunda rica asombr...
1,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_2,1,"Existen 14\nnacionalidades, 18 pueblos y 3 pue...",existen nacionalidades pueblos y pueblos indig...,existen nacionalidades pueblos pueblos indigen...
2,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_3,1,Por sus particulares características geográfic...,por sus particulares caracteristicas geografic...,particulares caracteristicas geograficas consi...
3,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_4,1,Su variada geografía con sus pisos climáticos ...,su variada geografia con sus pisos climaticos ...,variada geografia pisos climaticos hace produc...
4,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_5,1,"Desde los años setenta del siglo pasado, el Ec...",desde los anos setenta del siglo pasado el ecu...,anos setenta siglo pasado ecuador tambien expo...
...,...,...,...,...,...,...,...,...,...
11658,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_124_907,124,La rendición de cuentas será un mecanismo para...,la rendicion de cuentas sera un mecanismo para...,rendicion cuentas sera mecanismo evaluar dia d...
11659,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_125_908,125,Firmado digitalmente\n\nFirmado electrónicamen...,firmado digitalmente firmado electronicamente ...,firmado digitalmente firmado electronicamente ...
11660,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_125_909,125,Agradecemos a los expertos que han\naportado c...,agradecemos a los expertos que han aportado co...,agradecemos expertos aportado contenidos plan ...
11661,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_125_910,125,Se trata de un\ndocumento de autoría colectiva...,se trata de un documento de autoria colectiva ...,trata documento autoria colectiva susceptible ...


Función de eliminación de tíldes

In [70]:
def eliminar_tildes(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto)
                  if unicodedata.category(c) != 'Mn')

Función que tecteta si un texto en un encabezado o título numerado en documentos que tienen estructuras o índices.

In [71]:
def es_seccion_numerada(texto):
    # Patrones más específicos para detectar secciones numeradas
    patrones = [
        r'^\d+(\.\d+)*\.?\s+',  # Detecta "3. ", "3.1. ", etc.
        r'^\d+\.\s*$',          # Detecta "3." al final
        r'^\d+(\.\d+)+\s*$',    # Detecta "3.1" o "3.1.2" al final
        r'^\d+\s+\D',           # Detecta "3 Texto"
        r'.*\s+\d+(\.\d+)*\s*$' # Detecta "Texto 3.1"
    ]
    texto = texto.strip()
    return any(bool(re.match(patron, texto)) for patron in patrones)

Función que verifica si una sección de texto contiene numeración, al inicio y final del mismo

In [72]:
def es_seccion_numerada(texto):
    # Detecta números de sección al inicio o al final
    patrones = [
        r'^\d+(\.\d+)*\.',            # Detecta "1.", "1.2.", "1.2.3." al inicio
        r'.*\s+\d+(\.\d+)*\.$',       # Detecta números al final "texto 1.2."
        r'.*\s+\d+(\.\d+)*\s*$'       # Detecta números sin punto al final "texto 1.2"
    ]
    texto = texto.strip()
    return any(bool(re.search(patron, texto)) for patron in patrones)

Función que preprocesa y normaliza el texto eliminando tíldes, caracteres especiales y verificando secciones numeradas

In [ ]:
def limpiar_texto(texto):
    if not isinstance(texto, str):
        return ""
    
    # Verificar si es sección numerada antes de cualquier procesamiento
    if es_seccion_numerada(texto):
        return ""
    
    texto = texto.lower()
    texto = eliminar_tildes(texto)
    texto = re.sub(r'([^\d\w\s]|_)', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

Función que procesa el texto de la propuestade trabajo de un candidato, las separa en oraciones limpias, sin stop words y los estructura en dataframes

In [74]:
def procesar_plan_trabajo(row):
    resultados = []
    stopwords_spanish = set(stopwords.words('spanish'))
    
    texto = row['Plan_Trabajo']
    if not isinstance(texto, str):
        return pd.DataFrame()
    
    paginas = re.split(r'PAGINA \d+', texto)
    if paginas and not paginas[0].strip():
        paginas = paginas[1:]
    
    id_contador = 1
    
    for num_pagina, contenido_pagina in enumerate(paginas, 1):
        oraciones = sent_tokenize(contenido_pagina.strip())
        
        for oracion in oraciones:
            oracion = oracion.strip()
            if oracion and (oracion[0].isupper() or oracion.isupper()) and not es_seccion_numerada(oracion):
                oracion_limpia = limpiar_texto(oracion)
                
                if oracion_limpia:
                    palabras = word_tokenize(oracion_limpia)
                    sin_stopwords = ' '.join([palabra for palabra in palabras if palabra not in stopwords_spanish])
                    
                    resultados.append({
                        'Lista': row['Lista'],
                        'Partido': row['Partido'],
                        'Presidente': row['Presidente'],
                        'Vicepresidente': row['Vicepresidente'],
                        'id_oracion': f"{row['Lista']}_{num_pagina}_{id_contador}",
                        'pagina': num_pagina,
                        'oracion_original': oracion,
                        'oracion_limpia': oracion_limpia,
                        'oracion_sin_stopwords': sin_stopwords
                    })
                    id_contador += 1
    
    return pd.DataFrame(resultados)

Lectura del archivo dataframe_con_planes y lo carga en un DataFrame

In [75]:
print("Leyendo dataframe_con_planes.csv...")
df_planes = pd.read_csv("dataframe_con_planes.csv")

Leyendo dataframe_con_planes.csv...


Se procesa cada plan de trabajo en df_planes y guarda los resultados en una lista de DataFrames

In [76]:
dfs_procesados = []

print("Procesando planes de trabajo...")
for _, row in df_planes.iterrows():
    try:
        df_procesado = procesar_plan_trabajo(row)
        if not df_procesado.empty:
            dfs_procesados.append(df_procesado)
    except Exception as e:
        print(f"Error procesando plan de Lista {row['Lista']}: {e}")

Procesando planes de trabajo...


Combina los DataFrames procesados en un solo DataFrame y los guarda en un archivo CSV

In [77]:
df_final = pd.concat(dfs_procesados, ignore_index=True)
df_final.to_csv("planes_trabajo_procesados.csv", index=False, encoding='utf-8')

print("\nMuestra del DataFrame procesado:")
print(df_final.head(3))
print(f"\nTotal de oraciones procesadas: {len(df_final)}")


Muestra del DataFrame procesado:
   Lista                        Partido              Presidente  \
0      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   
1      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   
2      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   

            Vicepresidente id_oracion  pagina  \
0  LUCIA VALLECILLA SUAREZ      1_1_1       1   
1  LUCIA VALLECILLA SUAREZ      1_1_2       1   
2  LUCIA VALLECILLA SUAREZ      1_1_3       1   

                                    oracion_original  \
0  Introducción\nEcuador es un país con una profu...   
1  Existen 14\nnacionalidades, 18 pueblos y 3 pue...   
2  Por sus particulares características geográfic...   

                                      oracion_limpia  \
0  introduccion ecuador es un pais con una profun...   
1  existen 14 nacionalidades 18 pueblos y 3 puebl...   
2  por sus particulares caracteristicas geografic...   

                               oracion_sin_stopwor

In [78]:
df_final

,Lista,Partido,Presidente,Vicepresidente,id_oracion,pagina,oracion_original,oracion_limpia,oracion_sin_stopwords
0,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_1,1,Introducción\nEcuador es un país con una profu...,introduccion ecuador es un pais con una profun...,introduccion ecuador pais profunda rica asombr...
1,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_2,1,"Existen 14\nnacionalidades, 18 pueblos y 3 pue...",existen 14 nacionalidades 18 pueblos y 3 puebl...,existen 14 nacionalidades 18 pueblos 3 pueblos...
2,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_3,1,Por sus particulares características geográfic...,por sus particulares caracteristicas geografic...,particulares caracteristicas geograficas consi...
3,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_4,1,Su variada geografía con sus pisos climáticos ...,su variada geografia con sus pisos climaticos ...,variada geografia pisos climaticos hace produc...
4,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_5,1,"Desde los años setenta del siglo pasado, el Ec...",desde los anos setenta del siglo pasado el ecu...,anos setenta siglo pasado ecuador tambien expo...
...,...,...,...,...,...,...,...,...,...
11221,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_124_894,124,La rendición de cuentas será un mecanismo para...,la rendicion de cuentas sera un mecanismo para...,rendicion cuentas sera mecanismo evaluar dia d...
11222,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_125_895,125,Firmado digitalmente\n\nFirmado electrónicamen...,firmado digitalmente firmado electronicamente ...,firmado digitalmente firmado electronicamente ...
11223,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_125_896,125,Agradecemos a los expertos que han\naportado c...,agradecemos a los expertos que han aportado co...,agradecemos expertos aportado contenidos plan ...
11224,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_125_897,125,Se trata de un\ndocumento de autoría colectiva...,se trata de un documento de autoria colectiva ...,trata documento autoria colectiva susceptible ...


PREPROCESAMIENTO DE BIOGRAFIAS:

Eliminacion de tíldes

In [79]:
def eliminar_tildes(texto):
    return ''.join(c for c in unicodedata.normalize('NFD', texto)
                  if unicodedata.category(c) != 'Mn')

Función para determinar una sección numerada

In [80]:
def es_seccion_numerada(texto):
    patrones = [
        r'^\d+(\.\d+)*\.',            # Detecta "1.", "1.2.", "1.2.3." al inicio
        r'.*\s+\d+(\.\d+)*\.$',       # Detecta números al final "texto 1.2."
        r'.*\s+\d+(\.\d+)*\s*$'       # Detecta números sin punto al final "texto 1.2"
    ]
    texto = texto.strip()
    return any(bool(re.search(patron, texto)) for patron in patrones)


La función limpia y normaliza el texto, elimina caracteres especiales, convierte a minúsculas y elimina tildes. Se conserva porcentajes, números decimales y se decartan títulos de secciones numeradas de las biografías de los candidatos

In [81]:
def limpiar_texto(texto):
    if not isinstance(texto, str):
        return ""
    
    if es_seccion_numerada(texto):
        return ""
    
    # Convertir a minúsculas y eliminar tildes
    texto = texto.lower()
    texto = eliminar_tildes(texto)
    
    # Preservar porcentajes y años
    texto = re.sub(r'(\d+[\.,]?\d*)\s*%', r'\1 por ciento', texto)
    
    # Preservar números con punto decimal
    numeros_decimales = {}
    for i, match in enumerate(re.finditer(r'\d+\.\d+', texto)):
        placeholder = f'DECIMAL{i}'
        numeros_decimales[placeholder] = match.group()
        texto = texto.replace(match.group(), placeholder)
    
    # Eliminar puntuación pero preservar números
    texto = re.sub(r'([^\d\w\s]|_)', ' ', texto)
    
    # Restaurar números decimales
    for placeholder, numero in numeros_decimales.items():
        texto = texto.replace(placeholder, numero)
    
    # Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto)
    
    return texto.strip()

Función que procesa el texto de las biografpias de los candidatos, los divide en oraciones limpias, sin stopwords y los estructura en un DataFrame

In [82]:
def procesar_biografia(row):
    resultados = []
    stopwords_spanish = set(stopwords.words('spanish'))
    
    texto = row['Biografia']
    if not isinstance(texto, str):
        return pd.DataFrame()
    
    oraciones = sent_tokenize(texto.strip())
    id_contador = 1
    
    for oracion in oraciones:
        oracion = oracion.strip()
        if oracion and (oracion[0].isupper() or oracion.isupper()) and not es_seccion_numerada(oracion):
            oracion_limpia = limpiar_texto(oracion)
            
            if oracion_limpia:
                palabras = word_tokenize(oracion_limpia)
                sin_stopwords = ' '.join([palabra for palabra in palabras if palabra not in stopwords_spanish])
                
                resultados.append({
                    'Lista': row['Lista'],
                    'Partido': row['Partido'],
                    'Presidente': row['Presidente'],
                    'Vicepresidente': row['Vicepresidente'],
                    'id_oracion': f"{row['Lista']}_1_{id_contador}",
                    'oracion_original': oracion,
                    'oracion_limpia': oracion_limpia,
                    'oracion_sin_stopwords': sin_stopwords
                })
                id_contador += 1
    
    return pd.DataFrame(resultados)

Leer el archvico dataframe con las biografías y los carga en un DataFrame

In [83]:
print("Leyendo dataframe_con_biografias.csv...")
df_biografias = pd.read_csv("dataframe_con_biografias.csv")

Leyendo dataframe_con_biografias.csv...


Se procesan las biografías almacenadas aplicando la función procesar_biografia a cada fila y guardando los resultados en un DataFrame

In [84]:
dfs_procesados = []

print("Procesando biografías...")
for _, row in df_biografias.iterrows():
    try:
        df_procesado = procesar_biografia(row)
        if not df_procesado.empty:
            dfs_procesados.append(df_procesado)
    except Exception as e:
        print(f"Error procesando biografía de Lista {row['Lista']}: {e}")

Procesando biografías...


In [85]:
df_final = pd.concat(dfs_procesados, ignore_index=True)
df_final.to_csv("biografias_procesadas.csv", index=False, encoding='utf-8')

print("\nMuestra del DataFrame procesado:")
print(df_final.head(3))
print(f"\nTotal de oraciones procesadas: {len(df_final)}")


Muestra del DataFrame procesado:
   Lista                        Partido              Presidente  \
0      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   
1      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   
2      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   

            Vicepresidente id_oracion  \
0  LUCIA VALLECILLA SUAREZ      1_1_1   
1  LUCIA VALLECILLA SUAREZ      1_1_2   
2  LUCIA VALLECILLA SUAREZ      1_1_3   

                                    oracion_original  \
0  Jimmy Jairala y Lucía Vallecilla – Movimiento ...   
1  No registra títulos universitarios en la Secre...   
2  Su incursión en la política comenzó en 1992, c...   

                                      oracion_limpia  \
0  jimmy jairala y lucia vallecilla movimiento ce...   
1  no registra titulos universitarios en la secre...   
2  su incursion en la politica comenzo en 1992 cu...   

                               oracion_sin_stopwords  
0  jimmy jairala lucia vall

In [86]:
df_final

,Lista,Partido,Presidente,Vicepresidente,id_oracion,oracion_original,oracion_limpia,oracion_sin_stopwords
0,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_1,Jimmy Jairala y Lucía Vallecilla – Movimiento ...,jimmy jairala y lucia vallecilla movimiento ce...,jimmy jairala lucia vallecilla movimiento cent...
1,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_2,No registra títulos universitarios en la Secre...,no registra titulos universitarios en la secre...,registra titulos universitarios secretaria edu...
2,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_3,"Su incursión en la política comenzó en 1992, c...",su incursion en la politica comenzo en 1992 cu...,incursion politica comenzo 1992 subsecretario ...
3,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_4,En 2019 intentó nuevamente ser alcalde de Guay...,en 2019 intento nuevamente ser alcalde de guay...,2019 intento nuevamente ser alcalde guayaquil ...
4,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1_1_5,Centro Democrático ha sido una plataforma polí...,centro democratico ha sido una plataforma poli...,centro democratico sido plataforma politica ca...
...,...,...,...,...,...,...,...,...
271,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_1_11,Pensiones: Reformas al sistema de seguridad so...,pensiones reformas al sistema de seguridad social,pensiones reformas sistema seguridad social
272,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_1_12,Inversión: Alianzas público-privadas para el d...,inversion alianzas publico privadas para el de...,inversion alianzas publico privadas desarrollo...
273,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_1_13,"Carla Larrea\nCarla Larrea, de 46 años, es ing...",carla larrea carla larrea de 46 anos es ingeni...,carla larrea carla larrea 46 anos ingeniera co...
274,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,25_1_14,Su perfil de LinkedIn indica conocimientos en ...,su perfil de linkedin indica conocimientos en ...,perfil linkedin indica conocimientos mercado v...


PREPROCESAMIENTO DE ENTREVISTAS

Carga de stopwords en español

In [87]:
stopwords_spanish = set(stopwords.words('spanish'))

Función de sección numerada de documentos estructurados para las entrevistas

In [89]:
def es_seccion_numerada(texto):
    patron = r'^\d+(\.\d+)*\.?\s*[A-ZÁÉÍÓÚÑ]'
    return bool(re.match(patron, texto.strip()))

Función que procesa y normaliza el texto de las entrevistas, elimina caracteres especiales, convierte a minúsculas y elimina tildes

In [90]:
def limpiar_texto(texto):
    if not isinstance(texto, str):
        return ""
    
    texto = texto.lower()
    texto = eliminar_tildes(texto)
    texto = re.sub(r'([^\d\w\s]|_)', ' ', texto)
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

Función que elimina stopword del texto_limpio de las entrevistas

In [95]:
def get_sin_stopwords(texto_limpio):
    palabras = word_tokenize(texto_limpio)
    return ' '.join(palabra for palabra in palabras if palabra not in stopwords_spanish)

Función que extrae y separa las entrevistas de los candidatos presidenciales numeradas en un diccionario, la clave es el número de la entrevista y el valor su contenido

In [94]:
def separar_por_entrevistas(texto, tipo='ENTREVISTA'):
    if not isinstance(texto, str):
        return {}
    
    # Patrón para dividir por tipo de entrevista (ENTREVISTA, DESCRIPCION o TEMAS)
    patron = f"{tipo} (\d+)"
    partes = re.split(f"{patron}", texto)
    
    # El primer elemento es texto antes de cualquier marca, lo ignoramos si está vacío
    partes = partes[1:] if partes[0].strip() == '' else partes
    
    # Crear diccionario de entrevistas
    entrevistas = {}
    for i in range(0, len(partes)-1, 2):
        num_entrevista = int(partes[i])
        contenido = partes[i+1].strip()
        entrevistas[num_entrevista] = contenido
    
    return entrevistas

<>:6: SyntaxWarning: invalid escape sequence '\d'
<>:6: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Sloth\AppData\Local\Temp\ipykernel_16684\3063081743.py:6: SyntaxWarning: invalid escape sequence '\d'
  patron = f"{tipo} (\d+)"


Función que procesa las entrevistas, extre oraciones, elimina stopwords y las estructura en un DataFrame

In [91]:
def procesar_entrevista(row):
    resultados = []
    
    # Separar entrevistas, descripciones y temas
    entrevistas = separar_por_entrevistas(row['Entrevistas'], 'ENTREVISTA')
    descripciones = separar_por_entrevistas(row['Descripciones'], 'DESCRIPCION ENTREVISTA')
    temas = separar_por_entrevistas(row['Temas'], 'TEMAS ENTREVISTA')
    
    # Procesar cada entrevista
    for num_entrevista in entrevistas.keys():
        # Obtener texto correspondiente de cada tipo
        texto_entrevista = entrevistas.get(num_entrevista, '')
        texto_descripcion = descripciones.get(num_entrevista, '')
        texto_tema = temas.get(num_entrevista, '')
        
        # Procesar descripción y tema una vez por entrevista
        descripcion_limpia = limpiar_texto(texto_descripcion)
        tema_limpio = limpiar_texto(texto_tema)
        descripcion_sin_stopwords = get_sin_stopwords(descripcion_limpia)
        tema_sin_stopwords = get_sin_stopwords(tema_limpio)
        
        # Procesar oraciones de la entrevista
        if texto_entrevista:
            oraciones = sent_tokenize(texto_entrevista.strip())
            
            for id_contador, oracion in enumerate(oraciones, 1):
                oracion = oracion.strip()
                if oracion and (oracion[0].isupper() or oracion.isupper()) and not es_seccion_numerada(oracion):
                    oracion_limpia = limpiar_texto(oracion)
                    
                    if oracion_limpia:
                        sin_stopwords = get_sin_stopwords(oracion_limpia)
                        
                        resultados.append({
                            'Lista': row['Lista'],
                            'Partido': row['Partido'],
                            'Presidente': row['Presidente'],
                            'Vicepresidente': row['Vicepresidente'],
                            'numero_entrevista': num_entrevista,
                            'id_oracion': f"{row['Lista']}_{num_entrevista}_{id_contador}",
                            'oracion_original': oracion,
                            'oracion_limpia': oracion_limpia,
                            'oracion_sin_stopwords': sin_stopwords,
                            'descripcion_original': texto_descripcion,
                            'descripcion_limpia': descripcion_limpia,
                            'descripcion_sin_stopwords': descripcion_sin_stopwords,
                            'tema_original': texto_tema,
                            'tema_limpio': tema_limpio,
                            'tema_sin_stopwords': tema_sin_stopwords
                        })
    
    return pd.DataFrame(resultados)

Llama al archivo csv de las entrevistas y lo carga en un DataFrame

In [92]:
print("Leyendo dataframe_con_entrevistas.csv...")
df_entrevistas = pd.read_csv("dataframe_con_entrevistas.csv")

Leyendo dataframe_con_entrevistas.csv...


Procesamiento de las entrevistas almacenadas en el dataframe aplicando la función procesar_entrevista a cada fila y guardando los resultados en un dataframe

In [96]:
print("Procesando entrevistas...")
dfs_procesados = [procesar_entrevista(row) for _, row in df_entrevistas.iterrows()]
dfs_procesados = [df for df in dfs_procesados if not df.empty]

Procesando entrevistas...


Combinar los DataFrames procesados y los guarda en un archivo CSV

In [97]:
df_final = pd.concat(dfs_procesados, ignore_index=True)
df_final.to_csv("entrevistas_procesadas.csv", index=False, encoding='utf-8')

In [98]:
print("\nMuestra del DataFrame procesado:")
print(df_final.head(3))
print(f"\nTotal de oraciones procesadas: {len(df_final)}")


Muestra del DataFrame procesado:
   Lista                        Partido              Presidente  \
0      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   
1      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   
2      1  MOVIMIENTO CENTRO DEMOCRÁTICO  JIMMY JAIRALA VALLAZZA   

            Vicepresidente  numero_entrevista id_oracion  \
0  LUCIA VALLECILLA SUAREZ                  1      1_1_1   
1  LUCIA VALLECILLA SUAREZ                  1      1_1_2   
2  LUCIA VALLECILLA SUAREZ                  1      1_1_3   

                                    oracion_original  \
0  Entrevista a Jimmy Jairala – Ecuavisa\n\nEn es...   
1  Es su primer intento por llegar a Carondelet, ...   
2  La alcaldía de Guayaquil le fue esquiva en dos...   

                                      oracion_limpia  \
0  entrevista a jimmy jairala ecuavisa en esta es...   
1  es su primer intento por llegar a carondelet p...   
2  la alcaldia de guayaquil le fue esquiva en dos...   

      

In [99]:
# Mostrar estadísticas por número de entrevista
print("\nDistribución de oraciones por número de entrevista:")
print(df_final['numero_entrevista'].value_counts().sort_index())


Distribución de oraciones por número de entrevista:
numero_entrevista
1    1477
2    1438
3     759
Name: count, dtype: int64


In [100]:
df_final

,Lista,Partido,Presidente,Vicepresidente,numero_entrevista,id_oracion,oracion_original,oracion_limpia,oracion_sin_stopwords,descripcion_original,descripcion_limpia,descripcion_sin_stopwords,tema_original,tema_limpio,tema_sin_stopwords
0,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1,1_1_1,Entrevista a Jimmy Jairala – Ecuavisa\n\nEn es...,entrevista a jimmy jairala ecuavisa en esta es...,entrevista jimmy jairala ecuavisa especial ele...,"En esta entrevista, Jimmy Jairala, candidato p...",en esta entrevista jimmy jairala candidato pre...,entrevista jimmy jairala candidato presidencia...,"Trayectoria política, Centro Democrático, Nico...",trayectoria politica centro democratico nicola...,trayectoria politica centro democratico nicola...
1,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1,1_1_2,"Es su primer intento por llegar a Carondelet, ...",es su primer intento por llegar a carondelet p...,primer intento llegar carondelet postulante di...,"En esta entrevista, Jimmy Jairala, candidato p...",en esta entrevista jimmy jairala candidato pre...,entrevista jimmy jairala candidato presidencia...,"Trayectoria política, Centro Democrático, Nico...",trayectoria politica centro democratico nicola...,trayectoria politica centro democratico nicola...
2,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1,1_1_3,La alcaldía de Guayaquil le fue esquiva en dos...,la alcaldia de guayaquil le fue esquiva en dos...,alcaldia guayaquil esquiva dos ocasiones 2004 ...,"En esta entrevista, Jimmy Jairala, candidato p...",en esta entrevista jimmy jairala candidato pre...,entrevista jimmy jairala candidato presidencia...,"Trayectoria política, Centro Democrático, Nico...",trayectoria politica centro democratico nicola...,trayectoria politica centro democratico nicola...
3,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1,1_1_4,"En cambio, estuvo en la prefectura del Guayas ...",en cambio estuvo en la prefectura del guayas d...,cambio prefectura guayas dos periodos 2009 2018,"En esta entrevista, Jimmy Jairala, candidato p...",en esta entrevista jimmy jairala candidato pre...,entrevista jimmy jairala candidato presidencia...,"Trayectoria política, Centro Democrático, Nico...",trayectoria politica centro democratico nicola...,trayectoria politica centro democratico nicola...
4,1,MOVIMIENTO CENTRO DEMOCRÁTICO,JIMMY JAIRALA VALLAZZA,LUCIA VALLECILLA SUAREZ,1,1_1_5,"También ganó una curul como diputado, pero ese...",tambien gano una curul como diputado pero ese ...,tambien gano curul diputado cargo solo duro 10...,"En esta entrevista, Jimmy Jairala, candidato p...",en esta entrevista jimmy jairala candidato pre...,entrevista jimmy jairala candidato presidencia...,"Trayectoria política, Centro Democrático, Nico...",trayectoria politica centro democratico nicola...,trayectoria politica centro democratico nicola...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3669,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,3,25_3_62,"Finalmente, ¿qué hará con el Consejo de Partic...",finalmente que hara con el consejo de particip...,finalmente hara consejo participacion ciudadan...,"En esta entrevista, Henry Cucalón comparte su ...",en esta entrevista henry cucalon comparte su v...,entrevista henry cucalon comparte vision futur...,"Carrera política, Partido Social Cristiano, se...",carrera politica partido social cristiano sepa...,carrera politica partido social cristiano sepa...
3670,25,MOVIMIENTO CONSTRUYE,HENRY CUCALON,CARLA LARREA,3,25_3_63,Mi propuesta es reformar la Constitución.,mi propuesta es reformar la constitucion,propuesta reformar constitucion,"En esta entrevista, Henry Cucalón comparte su ...",en esta entrevista henry cucalon comparte su v...,entrevista henry cucalon comparte vision futur...,"Carrera política, Partido Social Cristiano, se...",carrera politica partido social cristiano sepa...,carrera politica partido 

# 2.Recuperación de información

In [2]:
!pip install numpy==1.25

     ---------------------------------------- 0.0/10.4 MB ? eta -:--:--
     -- ------------------------------------- 0.5/10.4 MB 4.2 MB/s eta 0:00:03
     ----- ---------------------------------- 1.3/10.4 MB 3.7 MB/s eta 0:00:03
     -------- ------------------------------- 2.1/10.4 MB 3.8 MB/s eta 0:00:03
     ------------ --------------------------- 3.1/10.4 MB 3.9 MB/s eta 0:00:02
     --------------- ------------------------ 3.9/10.4 MB 3.9 MB/s eta 0:00:02
     ----------------- ---------------------- 4.5/10.4 MB 3.5 MB/s eta 0:00:02
     -------------------- ------------------- 5.2/10.4 MB 3.6 MB/s eta 0:00:02
     ----------------------- ---------------- 6.0/10.4 MB 3.6 MB/s eta 0:00:02
     ------------------------- -------------- 6.6/10.4 MB 3.6 MB/s eta 0:00:02
     ---------------------------- ----------- 7.3/10.4 MB 3.5 MB/s eta 0:00:01
     ----------------------------- ---------- 7.6/10.4 MB 3.5 MB/s eta 0:00:01
     ------------------------------ --------- 7.9/10.4 MB 3

  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "C:\Users\Sloth\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 389, in <module>
          main()
        File "C:\Users\Sloth\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 373, in main
          json_out["return_val"] = hook(**hook_input["kwargs"])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "C:\Users\Sloth\AppData\Local\Programs\Python\Python312\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 137, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "C:\Users\Sloth\AppData\Local\Programs\Python\Python312\Lib\si

In [3]:
!pip install --upgrade scipy transformers sentence-transformers

  Using cached scipy-1.15.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached transformers-4.48.3-py3-none-any.whl.metadata (44 kB)
  Using cached sentence_transformers-3.4.1-py3-none-any.whl.metadata (10 kB)
Using cached scipy-1.15.1-cp312-cp312-win_amd64.whl (43.6 MB)
Using cached transformers-4.48.3-py3-none-any.whl (9.7 MB)
Using cached sentence_transformers-3.4.1-py3-none-any.whl (275 kB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.48.0
    Uninstalling transformers-4.48.0:
      Successfully uninstalled transformers-4.48.0
  Attempting uninstall: sentence-transformers
    Found existing installation: sentence-transformers 3.3.1
    Uninstalling sentence-transformers-3.3.1:
      Successfully uninstalled sentence-transformers-3.3.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.3 requires scipy<1.14.0,>=1.7.0, but you have scipy 1.15.1 which is incompatible.

[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: C:\Users\Sloth\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [10]:
#Importar las librerias necesarias
import pandas as pd
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer
import torch
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

Función para cargar los datos procesados de los archivos CSV: planes de trabajo, biografías y entrevistas.

In [11]:
def cargar_datos():
    """Cargar y preparar los datos de los tres archivos CSV"""
    print("Cargando datos...")
    planes = pd.read_csv('planes_trabajo_procesados.csv')
    biografias = pd.read_csv('biografias_procesadas.csv')
    entrevistas = pd.read_csv('entrevistas_procesadas.csv')

    return planes, biografias, entrevistas


Función que verifica si un texto está adecuado para la búsqueda, se descartan aquellos que sean demasiado cortos o que parezcan códigos o referencias.

In [ ]:
def es_texto_valido(texto):
    """Verifica si el texto es válido para búsqueda"""
    if not isinstance(texto, str):
        return False

    # Eliminar textos muy cortos
    if len(texto.split()) < 5:
        return False

    # Eliminar textos que son solo códigos o referencias
    if re.match(r'^[A-Z0-9\-\.]+$', texto.strip()):
        return False

    
    return True


La función descarta textos muy cortos y normaliza convirtiendo las palabras en minúsculas.

In [13]:
def preparar_texto_busqueda(texto):
    """Prepara el texto para búsqueda"""
    if not isinstance(texto, str):
        return ""
    palabras = texto.lower().split()
    if len(palabras) > 3:
        return texto
    return ""

Función que extrae fragmento del texto de una oración que sea relevante en un DataFrame, lo que proporciona un contexto de búsqueda.

In [14]:
def encontrar_contexto(df, idx, window=2):
    """Encuentra oraciones de contexto alrededor de un índice"""
    start_idx = max(0, idx - window)
    end_idx = min(len(df), idx + window + 1)

    #Extraer contexto de oraciones
    contexto = []
    for i in range(start_idx, end_idx):
        if i == idx:
            #Marca la oración como RELEVANTE
            contexto.append(f"[RELEVANTE] {df.iloc[i]['oracion_original']}")
        else:
            contexto.append(df.iloc[i]['oracion_original'])

    return " ".join(contexto)

Función que crea un sistema de búsqueda mejorado basado en TF-IDF, con embeddings de SentenceTransformer y FAISS.

In [ ]:
def crear_sistema_busqueda():
    """Crea el sistema de búsqueda mejorado"""
    planes, biografias, entrevistas = cargar_datos()

    # Preparar datos
    textos_validos = []
    metadata = []

    # Procesar planes de trabajo (peso más alto)
    for idx, row in planes.iterrows():
        if es_texto_valido(row['oracion_limpia']):
            textos_validos.append(row['oracion_limpia'])
            metadata.append({
                'tipo': 'plan',
                'peso': 1.2,  # Mayor peso para planes
                'lista': row['Lista'],
                'partido': row['Partido'],
                'presidente': row['Presidente'],
                'vicepresidente': row['Vicepresidente'],
                'texto_original': row['oracion_original'],
                'texto_contexto': encontrar_contexto(planes, idx),
                'id_oracion': row['id_oracion']
            })

    # Procesar entrevistas
    for idx, row in entrevistas.iterrows():
        if es_texto_valido(row['oracion_limpia']):
            textos_validos.append(row['oracion_limpia'])
            metadata.append({
                'tipo': 'entrevista',
                'peso': 1.1,  # Peso medio para entrevistas
                'lista': row['Lista'],
                'partido': row['Partido'],
                'presidente': row['Presidente'],
                'vicepresidente': row['Vicepresidente'],
                'numero_entrevista': row['numero_entrevista'],
                'texto_original': row['oracion_original'],
                'texto_contexto': encontrar_contexto(entrevistas, idx),
                'descripcion': row['descripcion_original'],
                'tema': row['tema_original'],
                'id_oracion': row['id_oracion']
            })

    # Procesar biografías (peso más bajo)
    for idx, row in biografias.iterrows():
        if es_texto_valido(row['oracion_limpia']):
            textos_validos.append(row['oracion_limpia'])
            metadata.append({
                'tipo': 'biografia',
                'peso': 1.0,  # Peso base para biografías
                'lista': row['Lista'],
                'partido': row['Partido'],
                'presidente': row['Presidente'],
                'vicepresidente': row['Vicepresidente'],
                'texto_original': row['oracion_original'],
                'texto_contexto': encontrar_contexto(biografias, idx),
                'id_oracion': row['id_oracion']
            })

    print(f"Total de textos válidos procesados: {len(textos_validos)}")

    # Crear vectorizador TF-IDF para pre-filtrado
    print("Creando vectorizador TF-IDF...")
    vectorizer = TfidfVectorizer(min_df=2, ngram_range=(1, 2))
    tfidf_matrix = vectorizer.fit_transform(textos_validos)

    # Crear modelo de embeddings
    print("Generando embeddings...")
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
    embeddings = model.encode(textos_validos, batch_size=32, show_progress_bar=True)

    # Crear índice FAISS
    print("Creando índice FAISS...")
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings.astype('float32'))


    """Devuelve un diccionario 
        index: índice FAISS para búsqueda semántica
        metadata: información de cada oración
        model: Modelo SentenceTransformer para nuevas consultas
        vectorizar: Modelo TF-IDF para búsqueda lexica
        tfidf_matrix: Representación TF-IDF de los textos
        textos_validos: lista de textos procesados."""

    return {
        'index': index,
        'metadata': metadata,
        'model': model,
        'vectorizer': vectorizer,
        'tfidf_matrix': tfidf_matrix,
        'textos_validos': textos_validos
    }

Función que realiza una búsqueda híbrida combinando TF-IDF y FAISS utilizando un umbral de similitud para filtrar resultados irrelevantes.

In [16]:
def buscar(query, sistema, k=5, umbral_similitud=0.3):
    """Realiza una búsqueda mejorada"""
    # Pre-filtrado con TF-IDF
    query_vec = sistema['vectorizer'].transform([query])
    similitudes = cosine_similarity(query_vec, sistema['tfidf_matrix']).flatten()

    # Obtener índices de documentos relevantes
    indices_relevantes = np.where(similitudes > umbral_similitud)[0]

    if len(indices_relevantes) == 0:
        return []

    # Generar embedding para la consulta
    query_embedding = sistema['model'].encode([query])

    # Buscar en FAISS
    index_subset = faiss.IndexFlatL2(sistema['index'].d)
    index_subset.add(sistema['index'].get_xb()[indices_relevantes])

    D, I = index_subset.search(query_embedding.astype('float32'), min(k, len(indices_relevantes)))

    # Mapear resultados y aplicar pesos
    resultados = []
    for i, (dist, idx) in enumerate(zip(D[0], I[0])):
        idx_original = indices_relevantes[idx]
        meta = sistema['metadata'][idx_original]

        dist_ajustada = dist / meta['peso']

        #Construcción de estructura de resultado
        resultado = {
            'ranking': i + 1,
            'distancia': dist_ajustada,
            'texto_original': meta['texto_original'],
            'texto_contexto': meta['texto_contexto'],
            'tipo': meta['tipo'],
            'lista': meta['lista'],
            'partido': meta['partido'],
            'presidente': meta['presidente'],
            'id_oracion': meta['id_oracion']
        }

        #Si el resultado es una entrevista, añade más información
        if meta['tipo'] == 'entrevista':
            resultado.update({
                'numero_entrevista': meta['numero_entrevista'],
                'descripcion': meta['descripcion'],
                'tema': meta['tema']
            })

        resultados.append(resultado)

    #Ordena los resultados de menor a mayor distancia
    #Retorna los resultados como una lista de diccionarios
    return sorted(resultados, key=lambda x: x['distancia'])


Crea , guarda y permite la búsqueda en el sistema de recuperación de información. Se utiliza FAISS, pickle y TF-IDF para optimizar la consulta.

In [17]:

if __name__ == "__main__":
    # Crear el sistema
    print("Creando sistema de búsqueda...")
    sistema = crear_sistema_busqueda()

    # Guardar el sistema
    print("Guardando sistema en sistema_busqueda.pkl...")
    sistema_para_guardar = {
        'index': faiss.serialize_index(sistema['index']),
        'metadata': sistema['metadata'],
        'vectorizer': sistema['vectorizer'],
        'tfidf_matrix': sistema['tfidf_matrix'],
        'textos_validos': sistema['textos_validos']
    }

    with open('sistema_busqueda.pkl', 'wb') as f:
        pickle.dump(sistema_para_guardar, f)

    print("Sistema guardado exitosamente!")

    def realizar_busqueda(query, k=5):
        """Función para realizar búsquedas"""
        print(f"\nBuscando: {query}")
        resultados = buscar(query, sistema, k)

        if not resultados:
            print("No se encontraron resultados relevantes.")
            return

        print("\nResultados encontrados:")
        for r in resultados:
            print(f"\nRanking: {r['ranking']}")
            print(f"Tipo: {r['tipo']}")
            print(f"Lista: {r['lista']} - Partido: {r['partido']}")
            print(f"Presidente: {r['presidente']}")
            print(f"ID: {r['id_oracion']}")
            print("\nContexto:")
            print(r['texto_contexto'])
            print(f"\nDistancia: {r['distancia']:.4f}")

            if r['tipo'] == 'entrevista':
                print(f"\nNúmero de entrevista: {r['numero_entrevista']}")
                print(f"Tema: {r['tema']}")
                print(f"Descripción: {r['descripcion']}")

    print("\nSistema listo para usar!")

Creando sistema de búsqueda...
Cargando datos...
Total de textos válidos procesados: 14820
Creando vectorizador TF-IDF...
Generando embeddings...


Batches:   0%|          | 0/464 [00:00<?, ?it/s]

Creando índice FAISS...
Guardando sistema en sistema_busqueda.pkl...
Sistema guardado exitosamente!

Sistema listo para usar!


#### Refinamiento del sistema

In [1]:
#importar librerías
import pickle
import faiss
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata
import re

Función para cargar el sistema de búsquesa desde el archivo sistema_busqueda.pkl, se evita así la reconstrucción desde cero.

In [2]:

def cargar_sistema():
    """Carga el sistema de búsqueda guardado"""
    print("Cargando sistema de búsqueda...")
    with open('sistema_busqueda.pkl', 'rb') as f:
        #pickle permite guardar y recuperar el sistema 
        #sin necesidad de recalcular modelos
        data = pickle.load(f)

    # FAISS no se puede almacenar directamente en pickle
    # por lo que se guarda en formato serializado

    data['index'] = faiss.deserialize_index(data['index'])
    return data

Función de normalización del texto: se eliminan tildes, convertir a minúsculas y limpiar caracteres especiales.

In [20]:
def normalizar_texto(texto):
    """Normaliza el texto: elimina tildes, convierte a minúsculas y elimina caracteres especiales"""
    if not isinstance(texto, str):
        return ""
    # Convertir a minúsculas
    texto = texto.lower()
    # Eliminar tildes
    texto = ''.join(c for c in unicodedata.normalize('NFD', texto)
                   if unicodedata.category(c) != 'Mn')
    # Eliminar caracteres especiales
    texto = re.sub(r'[^\w\s]', ' ', texto)
    # Eliminar espacios múltiples
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

Función que verifica si una query hace referencia a uno de los nombres de un candidato presidencial, compara palabras clave.

In [21]:

def es_nombre_presidente(query, presidente):
    """Verifica si la consulta corresponde al nombre del presidente"""
    # Normalizar tanto la consulta como el nombre del presidente
    query_norm = normalizar_texto(query)
    presidente_norm = normalizar_texto(presidente)

    # Obtener palabras normalizadas
    query_words = set(query_norm.split())
    presidente_words = set(presidente_norm.split())

    # Contar palabras coincidentes
    palabras_coincidentes = query_words.intersection(presidente_words)

    # Verificar si el nombre completo está contenido
    if query_norm in presidente_norm or presidente_norm in query_norm:
        return True

    # Verificar coincidencia de palabras individuales
    return len(palabras_coincidentes) >= 2

Función que asigna un puntaje de relevancia usando una lista de factores. Se da un peso especial a las biografías de los candidatos cuando la consulta menciona el nombre.

In [22]:
def calcular_relevancia(query, texto, meta):
    """Calcula un puntaje de relevancia incluyendo peso especial para biografías de presidentes"""
    # Factores de relevancia
    factores = {
        'palabras_clave': 0,
        'exactitud': 0,
        'tipo_doc': 0,
        'posicion': 0,
        'longitud': 0
    }

    # Normalizar textos para comparación
    texto_norm = normalizar_texto(texto)
    query_norm = normalizar_texto(query)
    query_words = set(query_norm.split())

    # Contar palabras clave encontradas
    palabras_encontradas = sum(1 for word in query_words if word in texto_norm)
    factores['palabras_clave'] = palabras_encontradas / len(query_words) if query_words else 0

    # Verificar coincidencia exacta de frases
    if query_norm in texto_norm:
        factores['exactitud'] = 1.0

    # Peso por tipo de documento con ajuste para biografías de presidentes
    pesos_tipo = {
        'plan': 1.3,
        'entrevista': 1.2,
        'biografia': 1.0
    }
    peso_base = pesos_tipo.get(meta['tipo'], 1.0)

    # Ajustar peso si es biografía del presidente buscado
    if meta['tipo'] == 'biografia' and es_nombre_presidente(query, meta['presidente']):
        peso_base *= 2.5  # Aumentar más el peso para biografías del presidente buscado
        factores['exactitud'] = 1.0  # Dar máxima exactitud si coincide el nombre

    factores['tipo_doc'] = peso_base

    # Posición de las palabras clave
    primera_aparicion = min((texto_norm.find(word) for word in query_words
                           if word in texto_norm), default=len(texto_norm))
    factores['posicion'] = 1.0 - (primera_aparicion / len(texto_norm))

    # Penalización por longitud
    palabras_texto = len(texto.split())
    if 10 <= palabras_texto <= 50:
        factores['longitud'] = 1.0
    else:
        factores['longitud'] = 0.8

    # Pesos para cada factor
    pesos = {
        'palabras_clave': 0.35,
        'exactitud': 0.25,
        'tipo_doc': 0.20,
        'posicion': 0.15,
        'longitud': 0.05
    }

    # Calcular puntaje final
    puntaje = sum(factor * pesos[nombre] for nombre, factor in factores.items())
    return puntaje

Función que realiza una búsqueda híbrida combinando filtrado inicial con TF_IDF, búsqueda semántica con SentenceTransformer y FAISS.

In [23]:
def buscar(query, k=5, umbral_similitud=0.3):
    """Realiza una búsqueda mejorada con mejor ranking"""
    sistema = cargar_sistema()

    # Pre-filtrado con TF-IDF
    query_vec = sistema['vectorizer'].transform([query])
    similitudes = cosine_similarity(query_vec, sistema['tfidf_matrix']).flatten()
    indices_relevantes = np.where(similitudes > umbral_similitud)[0]

    if len(indices_relevantes) == 0:
        return []

    # Búsqueda semántica con FAISS
    model = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')
    query_embedding = model.encode([query]).astype('float32')
    D, I = sistema['index'].search(query_embedding, k*2)  # Buscamos más resultados para filtrar

    # Preparar resultados con nuevo sistema de ranking
    resultados = []
    for i, (dist, idx) in enumerate(zip(D[0], I[0])):
        if idx < len(sistema['metadata']):
            meta = sistema['metadata'][idx]

            # Calcular relevancia personalizada
            relevancia = calcular_relevancia(
                query,
                meta['texto_original'],
                meta
            )

            # Ajustar distancia por relevancia
            dist_ajustada = dist / (relevancia + 0.1)  # Evitar división por cero

            resultado = {
                'distancia_original': dist,
                'relevancia': relevancia,
                'distancia_ajustada': dist_ajustada,
                'texto_original': meta['texto_original'],
                'texto_contexto': meta['texto_contexto'],
                'tipo': meta['tipo'],
                'lista': meta['lista'],
                'partido': meta['partido'],
                'presidente': meta['presidente'],
                'id_oracion': meta['id_oracion']
            }

            if meta['tipo'] == 'entrevista':
                resultado.update({
                    'numero_entrevista': meta['numero_entrevista'],
                    'descripcion': meta['descripcion'],
                    'tema': meta['tema']
                })

            resultados.append(resultado)

    # Ordenar por relevancia y distancia ajustada
    resultados = sorted(resultados,
                       key=lambda x: (-x['relevancia'], x['distancia_ajustada']))

    # Tomar los k mejores resultados
    return resultados[:k]


Función para mostrar el resultado de la búsqueda con un formato

In [24]:
def mostrar_resultados(query, k=5):
    """Muestra los resultados de búsqueda de manera formateada"""
    print(f"\nBuscando: {query}")
    resultados = buscar(query, k)

    if not resultados:
        print("No se encontraron resultados relevantes.")
        return

    print("\nResultados encontrados:")
    for i, r in enumerate(resultados, 1):
        print(f"\nRanking: {i}")
        print(f"Tipo: {r['tipo']}")
        print(f"Lista: {r['lista']} - Partido: {r['partido']}")
        print(f"Presidente: {r['presidente']}")
        print(f"ID: {r['id_oracion']}")
        print(f"Relevancia: {r['relevancia']:.4f}")
        print("\nContexto:")
        print(r['texto_contexto'])

        if r['tipo'] == 'entrevista':
            print(f"\nNúmero de entrevista: {r['numero_entrevista']}")
            print(f"Tema: {r['tema']}")
            print(f"Descripción: {r['descripcion']}")

In [19]:
if __name__ == "__main__":
    print("Sistema de búsqueda listo.")
    while True:
        print("\n" + "="*50)
        query = input("\nIngrese su búsqueda (o 'salir' para terminar): ")
        if query.lower() == 'salir':
            break

        k = input("Número de resultados a mostrar (Enter para usar 5): ")
        k = int(k) if k.strip() else 5

        mostrar_resultados(query, k)

Sistema de búsqueda listo.


Buscando: que candidato propone construir hospitales
Cargando sistema de búsqueda...

Resultados encontrados:

Ranking: 1
Tipo: plan
Lista: 2 - Partido: PARTIDO UNIDAD POPULAR
Presidente: JORGE ESCALA
ID: 2_25_256
Relevancia: 0.5619

Contexto:
Plan nacional de vacunación integral
Conformación de la industria farmacéutica nacional, que le permita a la población
tener acceso a medicamentos a precios accesibles. Cero tolerancia a la corrupción, los directores distritales y de hospitales, cada 3
meses deberán rendir informes a la comunidad, del manejo financiero de su
unidad y sector. [RELEVANTE] Se eliminarán los cargos burocráticos de gerencias hospitalarias que solo han
servido para inflar la nómina y como vectores de la corrupción, se rescatará la
dirección técnica de los hospitales con su Consejo Técnico Hospitalario, que serán
los responsables del manejo y orientación de cada hospital, encuadrados en el
sistema único nacional de salud. Rescataremos el SNE

## Evaluación del sistema

In [42]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score

Se carga el modelo de Sentence Transformer para convertir texto en vectores de embeddings para la evaluación

In [46]:
# Cargar el modelo de embeddings
modelo_embeddings = SentenceTransformer('paraphrase-multilingual-mpnet-base-v2')

Diccionario que almacena referencias textuales para evaluar las respuestas, se compara los decumentos recuperados con los textos de referencia.

In [45]:
# Ground Truth basado en textos de referencia más completos
ground_truth = {
    "reducción de impuestos": [
        "Propuestas de reducción de impuestos, reforma fiscal, eliminación de tributos innecesarios.",
        "Cómo mejorar la economía bajando los impuestos y ayudando a los contribuyentes."
    ],
    "seguridad ciudadana": [
        "Propuestas para mejorar la seguridad en el país, reducir la delincuencia y fortalecer la policía.",
        "Planes para combatir el crimen organizado y mejorar la protección ciudadana."
    ],
    "educación pública": [
        "Mejoras en la educación pública, acceso a la educación gratuita y reforma educativa.",
        "Planes para fortalecer el sistema educativo y garantizar educación de calidad."
    ],
    "empleo juvenil": [
        "Programas para fomentar el empleo juvenil, subsidios para jóvenes trabajadores.",
        "Propuestas para mejorar las oportunidades laborales para jóvenes."
    ],
    "cambio climático": [
        "Iniciativas para combatir el cambio climático, energías renovables y sostenibilidad.",
        "Planes para la protección del medio ambiente y políticas ecológicas."
    ]
}

Función que evalua la calidad del sistema de recuperación que utiliza embeddings de similitud semántica. Compara los documentos recuperados con el ground_truth y se calculan las métricas: precisión, recall y F1-score.

In [49]:
def evaluar_sistema_embeddings(queries, k=5):
    """Evalúa la eficacia del sistema usando embeddings de similitud semántica."""
    sistema = cargar_sistema()

    precision_scores = []
    recall_scores = []
    f1_scores = []

    for query in queries:
        print(f"\n🔎 Evaluando query: {query}")

        # Obtener los embeddings del ground truth de la consulta
        embeddings_gt = modelo_embeddings.encode(ground_truth[query])

        # Obtener resultados del sistema
        resultados = buscar(query, k)
        textos_obtenidos = [res['texto_original'] for res in resultados]

        # Si no hay resultados, precisión y recall son 0
        if not textos_obtenidos:
            print("❌ No se encontraron resultados.")
            precision_scores.append(0)
            recall_scores.append(0)
            f1_scores.append(0)
            continue

        # Obtener los embeddings de los textos obtenidos
        embeddings_resultados = modelo_embeddings.encode(textos_obtenidos)

        # Calcular similitudes entre cada resultado y el ground truth
        similitudes = cosine_similarity(embeddings_resultados, embeddings_gt)

        # Se considera relevante un documento si su similitud con algún texto del ground truth es mayor a 0.7
        y_true = [1 if np.max(similitud) > 0.7 else 0 for similitud in similitudes]
        y_pred = [1] * len(textos_obtenidos)

        # Mostrar resultados obtenidos y su similitud
        print("\n📌 **Documentos obtenidos y similitud:**")
        for i, (texto, similitud) in enumerate(zip(textos_obtenidos, similitudes)):
            max_sim = np.max(similitud)
            relevancia = "✅ Relevante" if max_sim > 0.7 else "❌ No relevante"
            print(f"{i+1}. {texto[:150]}... ({relevancia} - Similitud: {max_sim:.2f})")

        # Calcular métricas
        if sum(y_true) > 0:
            precision = precision_score(y_true, y_pred, zero_division=1)
            recall = recall_score(y_true, y_pred, zero_division=1)
            f1 = f1_score(y_true, y_pred, zero_division=1)
        else:
            precision, recall, f1 = 0, 0, 0  # Si no hay coincidencias, la métrica se mantiene en 0

        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)


    # 🔹 Promediar métricas
    avg_precision = np.mean(precision_scores)
    avg_recall = np.mean(recall_scores)
    avg_f1 = np.mean(f1_scores)

    print("\n📊 **Resultados Promedio**")
    print(f"📌 Precision@{k}: {avg_precision:.4f}")
    print(f"📌 Recall@{k}: {avg_recall:.4f}")
    print(f"📌 F1-Score@{k}: {avg_f1:.4f}")

In [50]:
# Ejecutar la evaluación con embeddings
queries_de_prueba = ["reducción de impuestos", "seguridad ciudadana", "educación pública", "empleo juvenil", "cambio climático"]
evaluar_sistema_embeddings(queries_de_prueba, k=5)

Cargando sistema de búsqueda...

🔎 Evaluando query: reducción de impuestos
Cargando sistema de búsqueda...

📌 **Documentos obtenidos y similitud:**
1. Sobre la economía y la reducción de impuestos
Ha propuesto reducir impuestos como el ISD.... (✅ Relevante - Similitud: 0.85)
2. Economía: Reducción de impuestos especiales.... (✅ Relevante - Similitud: 0.84)
3. Disminución del Impuesto a Consumos Especiales y del Impuesto a la
Salida de Divisas
Acciones:
o Realizar un análisis de impacto fiscal para determina... (✅ Relevante - Similitud: 0.73)
4. No más incrementos de impuestos ni amnistías tributarias: No incrementar más
impuestos, pero sí controlar los créditos tributarios y devoluciones, sin... (✅ Relevante - Similitud: 0.82)
5. Eliminación de impuestos como iSD, y simplificación de los
procedimientos de presentación de declaraciones y pago de impuestos.... (✅ Relevante - Similitud: 0.75)

🔎 Evaluando query: seguridad ciudadana
Cargando sistema de búsqueda...

📌 **Documentos obtenidos

# 3. Generación de respuestas con Ollama

In [25]:
#Importar librerías
import requests
import json
import re

In [ ]:
#  Frases irrelevantes que deben ser eliminadas antes de buscar
FRASES_IRRELEVANTES = [
    "que candidatos proponen",
    "qué candidatos proponen",
    "candidatos proponen",
    "qué candidatos plantean",
    "qué candidatos consideran",
    "qué candidatos tienen en su plan",
    "qué candidatos están a favor de",
    "que propone",
    "quién propone",
    "quienes proponen",
    "qué partido propone",
    "qué candidatos impulsan"
]

Función que elimina frases irrelevantes de la consulta del usuario antes de ejecutar la búsqueda.

In [27]:
def limpiar_query(query):
    """Limpia la consulta eliminando frases irrelevantes para mejorar la búsqueda."""
    query = query.lower().strip()
    
    # Eliminar frases completas irrelevantes
    for frase in FRASES_IRRELEVANTES:
        query = query.replace(frase, "").strip()

    # Eliminar espacios dobles después de limpiar
    query = re.sub(r'\s+', ' ', query)

    # Si después de limpiar la consulta está vacía, devolvemos la original
    return query if query else "No válida"

Función que genera una respuesta estructurada basada en los documentos recuperados de la búsqueda usando el modelo mistral a través de Ollama API.

In [28]:
def generar_respuesta_ollama(query, documentos):
    """Genera una respuesta estrictamente basada en los documentos obtenidos en la búsqueda."""

    if not documentos:
        return "No se encontraron documentos relevantes para generar una respuesta."

    # 🔹 Construcción de la lista de respuestas con información real
    ranking_info = []
    candidatos_info = []
    fuentes_info = []
    
    for i, doc in enumerate(documentos, 1):
        candidato = doc.get("presidente", "Candidato desconocido")
        partido = doc.get("partido", "Partido desconocido")
        tipo = doc.get("tipo", "Otro")
        relevancia = doc.get("relevancia", 0)
        distancia = doc.get("distancia_ajustada", 0)
        contexto = doc.get("texto_contexto", "").strip()

        # 🔹 Ranking inicial
        ranking_info.append(
            f"🔍 {candidato} - TIPO: {tipo} - RELEVANCIA: {relevancia:.4f} - DISTANCIA: {distancia:.4f}"
        )

        # 🔹 Información detallada del candidato
        candidatos_info.append(
            f"📌 {candidato} ({partido}):\n"
            f"Fuente: {tipo.capitalize()}\n"
            f"📄 Declaración: \"{contexto}\"\n"
        )

        # 🔹 Registro de la fuente completa
        fuentes_info.append(
            f"📚 {tipo.upper()} - {candidato} ({partido})\nTexto completo:\n{contexto}\n"
        )

    # 🔹 Construcción del prompt estructurado para Ollama
    prompt = f"""Genera una respuesta detallada y clara basada ÚNICAMENTE en la información recuperada:

💭 **Pregunta:** {query}

🔝 **Ranking final de los resultados:**
{chr(10).join(ranking_info)}

📝 **Respuesta:**
Se han encontrado los siguientes candidatos que mencionan la construcción de hospitales en sus propuestas:

{chr(10).join(candidatos_info)}

💡 **Análisis general:**  
De acuerdo con la información recuperada, se han identificado {len(documentos)} candidatos que incluyen la construcción o mejora de hospitales en sus propuestas. Algunas propuestas destacan la construcción de hospitales en zonas rurales, mientras que otras enfatizan la modernización de infraestructuras existentes o la mejora de la atención médica.

📚 **Fuentes principales consultadas:**
{chr(10).join(fuentes_info)}
"""

    try:
        # 🔹 Hacer la solicitud a Ollama sin streaming para recibir la respuesta completa más rápido
        respuesta = requests.post(
            "http://localhost:11434/api/generate",
            json={"model": "mistral", "prompt": prompt, "stream": False}
        )
        respuesta.raise_for_status()

        return respuesta.json().get("response", "No se pudo generar una respuesta.")

    except requests.exceptions.RequestException as e:
        return f"Error en la solicitud a Ollama: {e}"
    except ValueError as e:
        return f"Error al procesar la respuesta del modelo: {e}"



Función que realiza la búsqueda con el sistema RAG desarrollado y genera la respuesta basado en los documentos recuperados

In [29]:
def buscar_y_generar_respuesta(query, k=5):
    """Realiza la búsqueda con el sistema y genera una respuesta solo con documentos encontrados."""
    
    query_limpia = limpiar_query(query)
    print(f"🔍 Buscando: {query_limpia}")

    if query_limpia == "No válida":
        return "No se encontró una consulta válida para la búsqueda."

    # 🔹 Solo usamos documentos recuperados del corpus (SIN datos inventados)
    documentos = buscar(query_limpia, min(k, 5))  

    if not documentos:
        return "No se encontraron documentos relevantes para generar una respuesta."

    respuesta = generar_respuesta_ollama(query, documentos)

    return respuesta

Se ejecuta el sistema de búsqueda iterativamente a través de la terminal.

In [30]:
if __name__ == "__main__":
    print("🔎 Sistema de búsqueda optimizada con respuesta estructurada y basada en documentos.")

    while True:
        print("\n" + "="*50)
        query = input("\nIngrese su pregunta (o 'salir' para terminar): ")
        if query.lower() == 'salir':
            break

        k = input("Número de documentos a usar para la respuesta (Enter para usar 5): ")
        k = int(k) if k.strip() else 5

        respuesta = buscar_y_generar_respuesta(query, k)
        print("\n📢 Respuesta generada por el modelo:")
        print(respuesta)


🔎 Sistema de búsqueda optimizada con respuesta estructurada y basada en documentos.

🔍 Buscando: quien es francesco tabacchi
Cargando sistema de búsqueda...

📢 Respuesta generada por el modelo:
📚 PERFIL: FRANCESCO TABACCHI (MOVIMIENTO CREO, CREANDO OPORTUNIDADES)
  ---------------------------------------------------------------
  Nombre: Francesco Tabacchi
  Partido: Movimiento Creando Oportunidades (CREO)
  Edad: 53 años
  Profesión: Empresario ecuatoriano con amplia trayectoria en los sectores ganadero, agrícola y de la construcción.
  Cargos: Presidente de la Asociación de Ganaderos del Litoral y de la Federación de Ganaderos del Ecuador; Accionista y administrador de varias empresas registradas en la Superintendencia de Compañías, entre ellas: TR1 S.A.S.
  Fórmula política: Forma equipo con Blanca Saquicela, quien fue gobernadora del Guayas en 2023 y es candidata a prefecto de Guayas dentro del mismo movimiento Creo.

  📚 INTERVISTAS: FRANCESCO TABACCHI (MOVIMIENTO CREO, CREANDO OP